# Agent 驱动的金融建模系统 -- 方向A：自动化特征工程

## 结果索引
| 步骤 | 输出项 | 所在 Cell |
|------|--------|-----------|
| 1.1 | 数据读取与完整性验证 | Cell 4 |
| 1.2 | 数据概况报告 | Cell 6 |
| 1.3 | 可视化（收盘价趋势/缺失值热力图/标签分布） | Cell 8 |
| 1.4 | 数据泄漏初步检测 | Cell 10 |
| 2.1 | 逐特征诊断（缺失/异常/分布/相关性） | Cell 13 |
| 2.2 | 结构化特征诊断报告 | Cell 15 |
| 2.3 | 诊断可视化（缺失Top20/箱型图/相关性热力图） | Cell 17 |
| 3.1 | 千问Agent决策清理方案 | Cell 20 |
| 3.2 | 执行特征清理 (clean_features) | Cell 22 |
| 3.3 | 清理前后对比报告 | Cell 24 |
| 3.4 | 清理后数据验证 | Cell 26 |
| 4.1 | 特征有效性评估（Agent选标签+单特征LR） | Cell 29 |
| 4.2 | 特征冗余检测（相关系数+VIF） | Cell 31 |
| 4.3 | 综合特征评估报告与可视化 | Cell 33 |
| 5.1 | 三轮筛选 + Top50特征列表及入选理由 | Cell 36 |
| 5.2 | Agent选模型 + Top50模型训练验证 | Cell 38 |
| 6.1 | 综合可视化（数据概况+清理前后+特征评估） | Cell 41 |
| 6.2 | 模型评估可视化（混淆矩阵/ROC/PR曲线） | Cell 43 |
| 6.3 | 数据泄漏检测报告（独立模块, 5项检测） | Cell 45 |
| 6.4 | 完整报告汇总 + 产出文件索引 | Cell 47 |

---

## 步骤1：数据初始化与概况分析

In [1]:
import types as _types
if not hasattr(_types, 'UnionType'):
    _types.UnionType = type('UnionType', (), {})

import os
import sys
import warnings
import logging
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # 非交互后端，兼容无GUI环境
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# 设置中文字体和显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 120
plt.rcParams['savefig.dpi'] = 150

# pandas 显示设置
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

# 日志配置
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

warnings.filterwarnings('ignore')

# 常量定义
DATA_FILE = 'data.pq'
FEATURE_COLS = [f'X{i}' for i in range(1, 301)]   # X1 ~ X300
LABEL_COLS = [f'Y{i}' for i in range(1, 13)]       # Y1 ~ Y12
BASE_COLS = ['trade_date', 'underlying', 'start_time', 'end_time',
             'open', 'high', 'low', 'close', 'volume']
IMG_DIR = 'images'
os.makedirs(IMG_DIR, exist_ok=True)

print("[OK] 环境初始化完成")
print(f"Python: {sys.version}")
print(f"Pandas: {pd.__version__}, NumPy: {np.__version__}")

[OK] 环境初始化完成
Python: 3.10.0b4 (tags/v3.10.0b4:2ba4b20, Jul 10 2021, 17:36:48) [MSC v.1929 64 bit (AMD64)]
Pandas: 2.3.3, NumPy: 2.2.6


### 1.1 数据读取与完整性验证
读取 `data.pq` 文件，验证所有必需字段是否齐全（trade_date/underlying/start_time/end_time/open/high/low/close/volume/X1~X300/Y1~Y12）。

In [2]:
def load_and_validate_data(file_path: str) -> pd.DataFrame:
    """
    读取 parquet 格式的金融时序数据文件，并验证所有必需字段是否齐全。

    Description:
        1. 检查文件是否存在，获取文件大小。
        2. 使用 pyarrow 引擎读取 parquet 文件。
        3. 对比实际字段与预期字段（基础字段9个 + 特征字段X1~X300共300个
           + 标签字段Y1~Y12共12个 = 321个），输出验证摘要。

    Parameters:
        file_path : str
            parquet 文件的相对或绝对路径，标量字符串。

    Returns:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的 DataFrame，其中:
            - n_samples: 数据行数（样本数）
            - n_columns: 数据列数（至少321列：9基础 + 300特征 + 12标签）

    Raises:
        FileNotFoundError: 当指定路径的文件不存在时抛出。
        RuntimeError: 当 parquet 文件读取失败时抛出。
    """
    # 1. 验证文件存在性
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"[ERROR] 数据文件 '{file_path}' 不存在，请检查路径。")

    file_size_mb = os.path.getsize(file_path) / (1024 * 1024)
    logger.info(f"文件大小: {file_size_mb:.2f} MB")

    # 2. 读取 parquet 文件
    try:
        df = pd.read_parquet(file_path, engine='pyarrow')
        logger.info(f"数据读取成功: {df.shape[0]} 行 x {df.shape[1]} 列")
    except Exception as e:
        raise RuntimeError(f"[ERROR] 数据读取失败: {e}")

    # 3. 验证必需字段
    expected_cols = set(BASE_COLS + FEATURE_COLS + LABEL_COLS)
    actual_cols = set(df.columns)

    missing_cols = expected_cols - actual_cols
    extra_cols = actual_cols - expected_cols

    if missing_cols:
        logger.warning(f"[WARN] 缺失字段 ({len(missing_cols)}): {sorted(missing_cols)[:10]}...")
    else:
        logger.info("[OK] 所有必需字段验证通过")

    if extra_cols:
        logger.info(f"[INFO] 额外字段 ({len(extra_cols)}): {sorted(extra_cols)[:10]}")

    # 4. 输出验证摘要
    print("\n" + "="*60)
    print("数据完整性验证报告")
    print("="*60)
    print(f"  文件路径:     {file_path}")
    print(f"  文件大小:     {file_size_mb:.2f} MB")
    print(f"  数据规模:     {df.shape[0]:,} 行 x {df.shape[1]} 列")
    print(f"  必需字段数:   {len(expected_cols)}")
    print(f"  实际字段数:   {len(actual_cols)}")
    print(f"  缺失字段数:   {len(missing_cols)}")
    print(f"  额外字段数:   {len(extra_cols)}")

    # 列出每类字段的检查结果
    for name, cols in [("基础字段", BASE_COLS),
                       ("特征字段 X1~X300", FEATURE_COLS),
                       ("标签字段 Y1~Y12", LABEL_COLS)]:
        present = [c for c in cols if c in actual_cols]
        absent  = [c for c in cols if c not in actual_cols]
        status  = "[OK]" if not absent else "[FAIL]"
        print(f"  {status} {name}: {len(present)}/{len(cols)} 存在", end="")
        if absent:
            print(f"  缺失: {absent[:5]}")
        else:
            print()

    print("="*60)
    return df


# 执行数据加载
df = load_and_validate_data(DATA_FILE)

[2026-02-28 16:44:43,789] INFO: 文件大小: 173.68 MB
[2026-02-28 16:44:44,169] INFO: 数据读取成功: 81046 行 x 321 列
[2026-02-28 16:44:44,170] INFO: [OK] 所有必需字段验证通过



数据完整性验证报告
  文件路径:     data.pq
  文件大小:     173.68 MB
  数据规模:     81,046 行 x 321 列
  必需字段数:   321
  实际字段数:   321
  缺失字段数:   0
  额外字段数:   0
  [OK] 基础字段: 9/9 存在
  [OK] 特征字段 X1~X300: 300/300 存在
  [OK] 标签字段 Y1~Y12: 12/12 存在


### 1.2 数据概况报告
生成数据量、字段类型、时间范围、缺失值占比（按字段）、时序分布等汇总信息。

In [4]:
def generate_data_overview(df: pd.DataFrame) -> dict:
    """
    生成数据概况报告，包括基本统计、时间范围、缺失值分布和标签特征分析。

    Description:
        遍历所有字段，统计数据规模、内存占用、时间范围、各类字段的缺失率，
        并对 Y1~Y12 标签字段做值分布分析，为后续标签选择提供依据。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据，其中 n_columns 包含
            基础字段(9列)、特征字段X1~X300(300列)、标签字段Y1~Y12(12列)。

    Returns:
        report : dict
            包含以下键的字典:
            - 'shape': tuple (n_samples, n_columns), 数据维度
            - 'dtypes': dict, 各数据类型对应的列数
            - 'memory_mb': float, 内存占用(MB)
            - 'time_info': dict, 各时间字段的最小/最大值和唯一值数
            - 'missing': pd.DataFrame, 形状 (n_columns, 2), 含'缺失数'和'缺失率(%)'
            - 'label_stats': dict, Y1~Y12 各标签的统计信息
    """
    report = {}

    # --- 基本信息 ---
    report['shape'] = df.shape
    report['dtypes'] = df.dtypes.value_counts().to_dict()
    report['memory_mb'] = df.memory_usage(deep=True).sum() / (1024**2)

    # --- 时间范围 ---
    time_cols = ['trade_date', 'start_time', 'end_time']
    time_info = {}
    for col in time_cols:
        if col in df.columns:
            try:
                ts = pd.to_datetime(df[col])
                time_info[col] = {
                    'min': str(ts.min()),
                    'max': str(ts.max()),
                    'unique_count': ts.nunique()
                }
            except Exception:
                time_info[col] = {'error': '无法解析为时间类型'}
    report['time_info'] = time_info

    # --- 缺失值统计 ---
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        '缺失数': missing,
        '缺失率(%)': missing_pct
    }).sort_values('缺失率(%)', ascending=False)
    report['missing'] = missing_df

    # 按字段类型汇总缺失
    feature_cols_in_df = [c for c in FEATURE_COLS if c in df.columns]
    label_cols_in_df = [c for c in LABEL_COLS if c in df.columns]
    base_cols_in_df = [c for c in BASE_COLS if c in df.columns]

    feature_missing = missing_pct[feature_cols_in_df].describe()
    label_missing = missing_pct[label_cols_in_df].describe()
    base_missing = missing_pct[base_cols_in_df].describe()

    # --- 标签分布（用于自选Y的判断） ---
    label_stats = {}
    for col in LABEL_COLS:
        if col in df.columns:
            vc = df[col].value_counts(dropna=False)
            label_stats[col] = {
                'nunique': df[col].nunique(),
                'missing_pct': round(df[col].isnull().mean() * 100, 2),
                'value_counts': vc.head(10).to_dict(),
                'dtype': str(df[col].dtype)
            }
    report['label_stats'] = label_stats

    # --- 打印报告 ---
    print("\n" + "="*60)
    print("数据概况报告")
    print("="*60)
    print(f"\n[SIZE] 数据规模: {df.shape[0]:,} 行 x {df.shape[1]} 列")
    print(f"[MEM]  内存占用: {report['memory_mb']:.2f} MB")

    print(f"\n[DTYPE] 字段类型分布:")
    for dtype, cnt in report['dtypes'].items():
        print(f"   {dtype}: {cnt} 列")

    print(f"\n[TIME] 时间范围:")
    for col, info in time_info.items():
        if 'error' in info:
            print(f"   {col}: {info['error']}")
        else:
            print(f"   {col}: {info['min']} -> {info['max']} ({info['unique_count']} 个唯一值)")

    print(f"\n[MISSING] 缺失值概况:")
    print(f"   基础字段缺失率统计:\n{base_missing.to_string()}")
    print(f"\n   特征字段(X1~X300)缺失率统计:\n{feature_missing.to_string()}")
    print(f"\n   标签字段(Y1~Y12)缺失率统计:\n{label_missing.to_string()}")

    # 输出缺失率 > 0 的字段数
    has_missing = (missing_pct > 0).sum()
    high_missing = (missing_pct > 50).sum()
    print(f"\n   有缺失值的字段: {has_missing}/{len(df.columns)}")
    print(f"   缺失率 > 50% 的字段: {high_missing}/{len(df.columns)}")

    # 列出缺失率最高的 Top10 字段
    top_missing = missing_df[missing_df['缺失率(%)'] > 0].head(10)
    if len(top_missing) > 0:
        print(f"\n   缺失率 Top10 字段:")
        for idx, row in top_missing.iterrows():
            print(f"     {idx}: {row['缺失率(%)']:.2f}% ({int(row['缺失数']):,} 条)")

    print(f"\n[LABEL] 标签字段分析 (Y1~Y12):")
    for col, stats in label_stats.items():
        print(f"   {col}: dtype={stats['dtype']}, 唯一值={stats['nunique']}, "
              f"缺失率={stats['missing_pct']}%")
        # 如果是分类标签，显示类别分布
        if stats['nunique'] <= 20:
            vc_str = ", ".join([f"{k}:{v}" for k, v in list(stats['value_counts'].items())[:5]])
            print(f"         分布: {vc_str}")

    print("="*60)
    return report


# 执行
report = generate_data_overview(df)


数据概况报告

[SIZE] 数据规模: 81,046 行 x 321 列
[MEM]  内存占用: 202.43 MB

[DTYPE] 字段类型分布:
   float64: 317 列
   datetime64[us, Asia/Shanghai]: 2 列
   object: 1 列
   datetime64[ns]: 1 列

[TIME] 时间范围:
   trade_date: 2015-01-05 00:00:00 -> 2020-12-31 00:00:00 (1462 个唯一值)
   start_time: 2015-01-05 09:01:00+08:00 -> 2020-12-31 09:31:00+08:00 (5389 个唯一值)
   end_time: 2015-01-05 15:00:00+08:00 -> 2020-12-31 15:15:00+08:00 (2924 个唯一值)

[MISSING] 缺失值概况:
   基础字段缺失率统计:
count   9.0000
mean    0.0000
std     0.0000
min     0.0000
25%     0.0000
50%     0.0000
75%     0.0000
max     0.0000

   特征字段(X1~X300)缺失率统计:
count   300.0000
mean     23.6371
std       0.5740
min      23.2900
25%      23.2900
50%      23.3400
75%      23.3600
max      24.6800

   标签字段(Y1~Y12)缺失率统计:
count   12.0000
mean     0.0000
std      0.0000
min      0.0000
25%      0.0000
50%      0.0000
75%      0.0000
max      0.0000

   有缺失值的字段: 300/321
   缺失率 > 50% 的字段: 0/321

   缺失率 Top10 字段:
     X170: 24.68% (20,001 条)
     X200: 24.66% (19,986 

### 1.3 数据概况可视化
绘制三类关键图表：
1. **收盘价时间序列趋势图** — 按 `trade_date` 绘制 `close` 价格走势，观察时序分布与趋势特征；
2. **字段缺失值热力图** — 使用 missingno 对 X1~X300 特征字段绘制缺失矩阵，识别系统性缺失模式；
3. **标签(Y1~Y12)分布直方图** — 对所有标签字段绘制类别分布，为后续标签选择提供依据。

In [ ]:
def plot_close_price_trend(df: pd.DataFrame, save_dir: str = IMG_DIR) -> None:
    """
    绘制收盘价(close)随交易日期(trade_date)的时间序列趋势图。

    Description:
        1. 将 trade_date 解析为 datetime 类型并按日期排序。
        2. 按 trade_date 分组计算每日平均收盘价（同一天可能有多条记录）。
        3. 绘制折线图并标注关键统计量（最大/最小/均值）。
        4. 将图片保存至 save_dir/close_price_trend.png。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据，需包含 'trade_date' 和 'close' 列。
        save_dir : str
            图片保存目录路径，标量字符串，默认为 IMG_DIR。

    Returns:
        None
            无返回值，图片保存至文件并在 notebook 中显示。
    """
    fig, ax = plt.subplots(figsize=(14, 5))

    # 解析日期并按日聚合
    temp = df[['trade_date', 'close']].copy()
    temp['trade_date'] = pd.to_datetime(temp['trade_date'])
    daily = temp.groupby('trade_date')['close'].mean().sort_index()

    ax.plot(daily.index, daily.values, linewidth=0.8, color='steelblue', alpha=0.9)
    ax.axhline(daily.mean(), color='orange', linestyle='--', linewidth=0.8, label=f'均值={daily.mean():.2f}')
    ax.set_title('收盘价时间序列趋势图（按 trade_date 日均）', fontsize=13)
    ax.set_xlabel('交易日期')
    ax.set_ylabel('收盘价 (close)')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

    # 标注最大/最小
    max_date = daily.idxmax()
    min_date = daily.idxmin()
    ax.annotate(f'Max={daily.max():.2f}', xy=(max_date, daily.max()),
                xytext=(10, 10), textcoords='offset points', fontsize=8,
                arrowprops=dict(arrowstyle='->', color='red'), color='red')
    ax.annotate(f'Min={daily.min():.2f}', xy=(min_date, daily.min()),
                xytext=(10, -15), textcoords='offset points', fontsize=8,
                arrowprops=dict(arrowstyle='->', color='green'), color='green')

    plt.tight_layout()
    save_path = os.path.join(save_dir, 'close_price_trend.png')
    fig.savefig(save_path, bbox_inches='tight')
    plt.show()
    print(f"[OK] 收盘价趋势图已保存: {save_path}")
    print(f"     日期范围: {daily.index.min().date()} ~ {daily.index.max().date()}, 共 {len(daily)} 个交易日")


def plot_missing_heatmap(df: pd.DataFrame, feature_cols: list, save_dir: str = IMG_DIR) -> None:
    """
    绘制特征字段(X1~X300)的缺失值矩阵热力图。

    Description:
        1. 从 df 中提取 feature_cols 对应的列。
        2. 使用 missingno.matrix 绘制缺失值可视化矩阵，白色表示缺失。
        3. 同时绘制一张按特征排序的缺失率柱状图作为补充。
        4. 将图片保存至 save_dir/missing_heatmap.png 和 missing_bar.png。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据。
        feature_cols : list of str
            特征列名列表，长度为 n_features（最多300），如 ['X1', 'X2', ..., 'X300']。
        save_dir : str
            图片保存目录路径，标量字符串，默认为 IMG_DIR。

    Returns:
        None
            无返回值，图片保存至文件并在 notebook 中显示。
    """
    cols_in_df = [c for c in feature_cols if c in df.columns]

    # 1) missingno 矩阵图 — 抽样50个特征以保持可读性
    sample_cols = cols_in_df[::6]  # 每隔6个取1个，约50个特征
    fig1 = msno.matrix(df[sample_cols], figsize=(16, 6), sparkline=False, fontsize=7)
    fig1.set_title('特征字段缺失值矩阵 (X1~X300 抽样)', fontsize=13)
    save_path1 = os.path.join(save_dir, 'missing_heatmap.png')
    fig1.get_figure().savefig(save_path1, bbox_inches='tight')
    plt.show()
    print(f"[OK] 缺失值矩阵图已保存: {save_path1}")

    # 2) 缺失率柱状图
    missing_pct = df[cols_in_df].isnull().mean() * 100
    fig2, ax2 = plt.subplots(figsize=(16, 4))
    ax2.bar(range(len(missing_pct)), missing_pct.values, width=1.0, color='coral', alpha=0.7)
    ax2.axhline(missing_pct.mean(), color='navy', linestyle='--', linewidth=0.8,
                label=f'平均缺失率={missing_pct.mean():.2f}%')
    ax2.set_title('特征字段缺失率分布 (X1~X300)', fontsize=13)
    ax2.set_xlabel('特征编号')
    ax2.set_ylabel('缺失率 (%)')
    ax2.set_xlim(-1, len(missing_pct))
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')
    # 标注刻度: 每50个
    tick_pos = list(range(0, len(cols_in_df), 50))
    tick_labels = [cols_in_df[i] for i in tick_pos]
    ax2.set_xticks(tick_pos)
    ax2.set_xticklabels(tick_labels, fontsize=8)
    plt.tight_layout()
    save_path2 = os.path.join(save_dir, 'missing_bar.png')
    fig2.savefig(save_path2, bbox_inches='tight')
    plt.show()
    print(f"[OK] 缺失率柱状图已保存: {save_path2}")


def plot_all_label_distributions(df: pd.DataFrame, label_cols: list,
                                  save_dir: str = IMG_DIR) -> None:
    """
    绘制所有标签字段(Y1~Y12)的类别分布子图，为后续标签选择提供可视化依据。

    Description:
        1. 在 3x4 网格中为每个标签绘制柱状图，展示各类别的样本数与占比。
        2. 在每个子图标题中标注唯一值数和缺失率。
        3. 评估每个标签的类别不平衡比并汇总输出。
        4. 将图片保存至 save_dir/label_distributions_all.png。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据，需包含 Y1~Y12 标签列。
        label_cols : list of str
            标签列名列表，长度为 12，如 ['Y1', 'Y2', ..., 'Y12']。
        save_dir : str
            图片保存目录路径，标量字符串，默认为 IMG_DIR。

    Returns:
        None
            无返回值，图片保存至文件并在 notebook 中显示。
    """
    cols_in_df = [c for c in label_cols if c in df.columns]
    n_labels = len(cols_in_df)
    ncols = 4
    nrows = (n_labels + ncols - 1) // ncols
    colors = ['#4CAF50', '#FF9800', '#F44336', '#2196F3', '#9C27B0', '#795548']

    fig, axes = plt.subplots(nrows, ncols, figsize=(18, 4 * nrows))
    axes = axes.flatten()

    summary_lines = []

    for i, col in enumerate(cols_in_df):
        ax = axes[i]
        s = df[col].dropna()
        vc = s.value_counts().sort_index()
        total = len(s)
        missing_pct = df[col].isnull().mean() * 100

        bars = ax.bar([str(v) for v in vc.index], vc.values,
                      color=colors[:len(vc)], alpha=0.85, edgecolor='black', linewidth=0.3)

        # 柱顶标注百分比
        for bar, (val, cnt) in zip(bars, vc.items()):
            pct = cnt / total * 100
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
                    f'{pct:.0f}%', ha='center', va='bottom', fontsize=7)

        ax.set_title(f'{col} (唯一值={vc.nunique()}, 缺失={missing_pct:.1f}%)', fontsize=9)
        ax.set_xlabel('')
        ax.set_ylabel('样本数' if i % ncols == 0 else '')
        ax.grid(True, alpha=0.2, axis='y')
        ax.tick_params(axis='both', labelsize=7)

        # 汇总信息
        imbalance_ratio = vc.max() / vc.min() if vc.min() > 0 else float('inf')
        summary_lines.append(
            f"  {col}: 唯一值={vc.nunique()}, 缺失率={missing_pct:.2f}%, "
            f"不平衡比={imbalance_ratio:.2f}:1"
        )

    # 隐藏多余子图
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    fig.suptitle('标签字段 Y1~Y12 类别分布总览', fontsize=14, y=1.01)
    plt.tight_layout()
    save_path = os.path.join(save_dir, 'label_distributions_all.png')
    fig.savefig(save_path, bbox_inches='tight')
    plt.show()
    print(f"[OK] 标签分布总览图已保存: {save_path}")

    # 输出汇总
    print(f"\n[SUMMARY] 各标签统计:")
    for line in summary_lines:
        print(line)


# ---- 执行可视化 ----
print("=" * 60)
print("步骤 1.3: 数据概况可视化")
print("=" * 60)

print("\n[1/3] 收盘价时间序列趋势图")
plot_close_price_trend(df)

print("\n[2/3] 特征字段缺失值热力图")
plot_missing_heatmap(df, FEATURE_COLS)

print("\n[3/3] 标签字段 Y1~Y12 分布直方图")
plot_all_label_distributions(df, LABEL_COLS)

步骤 1.4: 数据概况可视化

[1/3] 收盘价时间序列趋势图
[OK] 收盘价趋势图已保存: images\close_price_trend.png
     日期范围: 2015-01-05 ~ 2020-12-31, 共 1462 个交易日

[2/3] 特征字段缺失值热力图
[OK] 缺失值矩阵图已保存: images\missing_heatmap.png


[2026-02-28 16:40:12,694] INFO: Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
[2026-02-28 16:40:12,695] INFO: Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.


[OK] 缺失率柱状图已保存: images\missing_bar.png

[3/3] 目标标签 Y4 分布直方图
[OK] 标签分布图已保存: images\label_distribution.png
     总有效样本: 81,046, 类别数: 3
     最大类占比: 40.8%, 最小类占比: 19.8%
     不平衡比: 2.07:1
     [OK] 类别分布相对均衡


### 1.4 数据泄漏初步检测
检测以下潜在的数据泄漏风险：
1. **时序字段混乱** — 检查是否存在 `start_time > end_time` 的异常记录；
2. **时间逻辑错误** — 检查 `trade_date` 与 `start_time`/`end_time` 的日期是否一致；
3. **特征-标签极端相关性** — 对 Y1~Y12 所有标签，计算特征的 Pearson 相关系数，识别 |r| > 0.95 的可疑特征；
4. **未来数据泄漏** — 检查同一 underlying 下是否存在时间交叉或乱序情况。

In [3]:

def check_time_field_logic(df: pd.DataFrame) -> dict:
    """
    检测时序字段的逻辑一致性，识别时间混乱和日期不匹配的记录。

    Description:
        1. 检查 start_time > end_time 的异常记录数。
        2. 检查 trade_date 的日期部分是否与 start_time 的日期部分一致。
        3. 检查同一 underlying 内时间序列是否严格递增。
        4. 返回检测结果汇总字典。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据，需包含
            'trade_date', 'start_time', 'end_time', 'underlying' 列。

    Returns:
        result : dict
            包含以下键的检测结果字典:
            - 'start_gt_end_count': int, start_time > end_time 的记录数
            - 'date_mismatch_count': int, trade_date 与 start_time 日期不一致的记录数
            - 'time_not_sorted_groups': int, 时间未严格排序的 underlying 分组数
            - 'total_groups': int, underlying 分组总数
            - 'issues': list of str, 发现的问题描述列表
    """
    result = {
        'start_gt_end_count': 0,
        'date_mismatch_count': 0,
        'time_not_sorted_groups': 0,
        'total_groups': 0,
        'issues': []
    }

    # 解析时间字段
    try:
        start_ts = pd.to_datetime(df['start_time'])
        end_ts = pd.to_datetime(df['end_time'])
        trade_dt = pd.to_datetime(df['trade_date'])
    except Exception as e:
        result['issues'].append(f"时间字段解析失败: {e}")
        return result

    # 1. 检查 start_time > end_time
    bad_order = start_ts > end_ts
    result['start_gt_end_count'] = int(bad_order.sum())
    if result['start_gt_end_count'] > 0:
        result['issues'].append(
            f"发现 {result['start_gt_end_count']} 条记录 start_time > end_time"
        )
        bad_idx = df[bad_order].index[:5]
        print(f"  [WARN] start_time > end_time 异常示例 (前5条):")
        for idx in bad_idx:
            print(f"    行{idx}: start={df.loc[idx, 'start_time']}, end={df.loc[idx, 'end_time']}")

    # 2. 检查 trade_date 与 start_time 日期是否一致
    start_date = start_ts.dt.date
    trade_date = trade_dt.dt.date
    date_mismatch = start_date != trade_date
    result['date_mismatch_count'] = int(date_mismatch.sum())
    if result['date_mismatch_count'] > 0:
        result['issues'].append(
            f"发现 {result['date_mismatch_count']} 条记录 trade_date 与 start_time 日期不一致"
        )

    # 3. 检查同一 underlying 内 start_time 是否递增
    if 'underlying' in df.columns:
        groups = df.groupby('underlying')
        result['total_groups'] = groups.ngroups
        not_sorted = 0
        for name, group in groups:
            ts = pd.to_datetime(group['start_time'])
            if not ts.is_monotonic_increasing:
                not_sorted += 1
        result['time_not_sorted_groups'] = not_sorted
        if not_sorted > 0:
            result['issues'].append(
                f"发现 {not_sorted}/{result['total_groups']} 个 underlying 分组时间未严格递增"
            )

    return result


def check_feature_label_leakage(df: pd.DataFrame, feature_cols: list,
                                 label_cols: list, threshold: float = 0.95) -> dict:
    """
    检测特征与所有标签(Y1~Y12)之间是否存在极端相关性（疑似数据泄漏）。

    Description:
        1. 对每个标签列，计算所有特征的 Pearson 相关系数（忽略缺失值）。
        2. 筛选相关系数绝对值 > threshold 的可疑特征。
        3. 返回各标签的可疑特征列表及相关系数汇总。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据。
        feature_cols : list of str
            特征列名列表，长度最多为 300，如 ['X1', 'X2', ..., 'X300']。
        label_cols : list of str
            标签列名列表，长度最多为 12，如 ['Y1', 'Y2', ..., 'Y12']。
        threshold : float
            相关系数绝对值阈值，标量浮点数，默认为 0.95。

    Returns:
        result : dict
            包含以下键的字典:
            - 'by_label': dict, 键为标签名，值为 pd.DataFrame (n_suspicious, 2)
              含 'feature' 和 'correlation' 列
            - 'all_correlations': pd.DataFrame, 形状为 (n_features, n_labels),
              所有特征与所有标签的相关系数矩阵
            - 'total_suspicious': int, 全部可疑（特征,标签）对数
    """
    cols_in_df = [c for c in feature_cols if c in df.columns]
    labels_in_df = [c for c in label_cols if c in df.columns]

    result = {'by_label': {}, 'total_suspicious': 0}

    # 计算完整相关系数矩阵
    all_corr_data = {}
    for label in labels_in_df:
        corrs = df[cols_in_df].corrwith(df[label]).dropna()
        all_corr_data[label] = corrs

        # 筛选可疑特征
        suspicious_mask = corrs.abs() > threshold
        if suspicious_mask.any():
            suspicious = pd.DataFrame({
                'feature': corrs[suspicious_mask].index,
                'correlation': corrs[suspicious_mask].values
            }).sort_values('correlation', key=abs, ascending=False).reset_index(drop=True)
            result['by_label'][label] = suspicious
            result['total_suspicious'] += len(suspicious)

    result['all_correlations'] = pd.DataFrame(all_corr_data)
    return result


def run_leakage_detection(df: pd.DataFrame, feature_cols: list, label_cols: list) -> dict:
    """
    执行完整的数据泄漏初步检测流程。

    Description:
        汇总运行时序逻辑检查和特征-标签(Y1~Y12)极端相关性检查，输出综合报告。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据。
        feature_cols : list of str
            特征列名列表，长度最多为 300。
        label_cols : list of str
            标签列名列表，长度最多为 12。

    Returns:
        leakage_report : dict
            包含以下键的字典:
            - 'time_check': dict, 时序逻辑检查结果
            - 'correlation_check': dict, 特征-标签极端相关性检查结果
            - 'has_issues': bool, 是否发现任何问题
    """
    leakage_report = {}

    # ---- 1. 时序逻辑检查 ----
    print("[1/2] 时序字段逻辑检查")
    print("-" * 40)
    time_result = check_time_field_logic(df)
    leakage_report['time_check'] = time_result

    print(f"  start_time > end_time 异常记录:    {time_result['start_gt_end_count']}")
    print(f"  trade_date 与 start_time 日期不一致: {time_result['date_mismatch_count']}")
    if time_result['total_groups'] > 0:
        print(f"  时间未排序的 underlying 分组:      "
              f"{time_result['time_not_sorted_groups']}/{time_result['total_groups']}")

    if not time_result['issues']:
        print("  [OK] 时序字段逻辑检查通过，未发现异常")
    else:
        for issue in time_result['issues']:
            print(f"  [WARN] {issue}")

    # ---- 2. 特征-标签极端相关性检查 ----
    print(f"\n[2/2] 特征与标签 Y1~Y12 的极端相关性检查 (阈值 |r| > 0.95)")
    print("-" * 40)

    corr_result = check_feature_label_leakage(df, feature_cols, label_cols, threshold=0.95)
    leakage_report['correlation_check'] = corr_result

    if corr_result['total_suspicious'] == 0:
        print(f"  [OK] 未发现与任何标签相关系数绝对值 > 0.95 的特征")
    else:
        print(f"  [WARN] 共发现 {corr_result['total_suspicious']} 对可疑（特征,标签）组合:")
        for label, sus_df in corr_result['by_label'].items():
            for _, row in sus_df.iterrows():
                print(f"    {row['feature']} <-> {label}: r = {row['correlation']:.4f}")

    # 汇总各标签的相关系数分布
    corr_df = corr_result['all_correlations']
    print(f"\n  特征-标签相关系数分布统计 (各标签):")
    print(f"  {'标签':<6} {'|r|>0.9':>8} {'|r|>0.5':>8} {'|r|<0.01':>9} {'均值':>8} {'标准差':>8}")
    print(f"  {'-'*50}")
    for label in corr_df.columns:
        c = corr_df[label].dropna()
        print(f"  {label:<6} {(c.abs() > 0.9).sum():>8} {(c.abs() > 0.5).sum():>8} "
              f"{(c.abs() < 0.01).sum():>9} {c.mean():>8.4f} {c.std():>8.4f}")

    # 绘制相关系数热力图 — 挑选3个代表性标签
    sample_labels = corr_df.columns[:3].tolist()  # Y1, Y2, Y3
    fig, axes = plt.subplots(1, len(sample_labels), figsize=(16, 4))
    if len(sample_labels) == 1:
        axes = [axes]
    for ax, label in zip(axes, sample_labels):
        c = corr_df[label].dropna()
        ax.hist(c.values, bins=50, color='steelblue', alpha=0.7, edgecolor='black', linewidth=0.3)
        ax.axvline(0, color='red', linestyle='--', linewidth=0.8)
        ax.axvline(0.95, color='orange', linestyle='--', linewidth=0.8)
        ax.axvline(-0.95, color='orange', linestyle='--', linewidth=0.8)
        ax.set_title(f'特征 vs {label} 相关系数', fontsize=10)
        ax.set_xlabel('Pearson r')
        ax.set_ylabel('特征数')
        ax.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    save_path = os.path.join(IMG_DIR, 'correlation_distribution.png')
    fig.savefig(save_path, bbox_inches='tight')
    plt.show()
    print(f"\n  [OK] 相关系数分布图已保存: {save_path}")

    # 综合判断
    has_issues = bool(time_result['issues']) or corr_result['total_suspicious'] > 0
    leakage_report['has_issues'] = has_issues

    print(f"\n{'='*60}")
    if has_issues:
        print("[WARN] 数据泄漏检测发现潜在问题，请关注上述警告")
    else:
        print("[OK] 数据泄漏初步检测通过，未发现明显风险")
    print(f"{'='*60}")

    return leakage_report


# ---- 执行数据泄漏检测 ----
print("=" * 60)
print("步骤 1.4: 数据泄漏初步检测")
print("=" * 60 + "\n")

leakage_report = run_leakage_detection(df, FEATURE_COLS, LABEL_COLS)

步骤 1.4: 数据泄漏初步检测

[1/2] 时序字段逻辑检查
----------------------------------------
  start_time > end_time 异常记录:    0
  trade_date 与 start_time 日期不一致: 43978
  时间未排序的 underlying 分组:      0/70
  [WARN] 发现 43978 条记录 trade_date 与 start_time 日期不一致

[2/2] 特征与标签 Y1~Y12 的极端相关性检查 (阈值 |r| > 0.95)
----------------------------------------
  [OK] 未发现与任何标签相关系数绝对值 > 0.95 的特征

  特征-标签相关系数分布统计 (各标签):
  标签      |r|>0.9  |r|>0.5  |r|<0.01       均值      标准差
  --------------------------------------------------
  Y1            0        0       101   0.0166   0.0398
  Y2            0        0       115   0.0140   0.0346
  Y3            0        0       129   0.0091   0.0259
  Y4            0        0       182   0.0021   0.0157
  Y5            0        0        99   0.0181   0.0412
  Y6            0        0       113   0.0142   0.0346
  Y7            0        0       148   0.0093   0.0256
  Y8            0        0       188   0.0032   0.0141
  Y9            0        0        94   0.0167   0.0402
  Y10           0  

## 步骤2：自动特征诊断

对 X1~X300 共 300 个特征逐字段进行全面诊断，内容包括：
- **缺失值分析**：缺失比例、缺失模式（随机/时序连续缺失）
- **异常值检测**：IQR 法 + 3-sigma 检测，兼顾时序趋势避免误判
- **分布特征**：偏度/峰度、正态性检验（Shapiro-Wilk 抽样）、ADF 平稳性检验
- **与标签相关性**：所有 Y1~Y12 的 Pearson/Spearman 相关系数及显著性

### 2.1 逐特征诊断：缺失值 / 异常值 / 分布 / 相关性
遍历 X1~X300，为每个特征生成完整的诊断记录，汇总到 `diag_records` 列表中。

In [6]:
# =============================================================================
# Cell: 逐特征诊断 — 缺失值 / 异常值 / 分布 / 相关性
# =============================================================================
from scipy import stats as sp_stats
from statsmodels.tsa.stattools import adfuller


def diagnose_missing(series: pd.Series) -> dict:
    """
    诊断单个特征的缺失值情况，包括缺失比例和缺失模式。

    Description:
        1. 计算缺失数量和缺失比例。
        2. 通过连续缺失段长度判断缺失模式:
           - 最长连续缺失段 >= 50 且占总缺失 >= 30% 视为 "时序连续缺失"
           - 否则视为 "随机缺失"

    Parameters:
        series : pd.Series
            形状为 (n_samples,) 的单列特征数据。

    Returns:
        result : dict
            - 'missing_count': int, 缺失值数量
            - 'missing_pct': float, 缺失比例(%)
            - 'missing_pattern': str, '随机缺失' 或 '时序连续缺失' 或 '无缺失'
            - 'max_consecutive_missing': int, 最长连续缺失段长度
    """
    n = len(series)
    missing_count = int(series.isnull().sum())
    missing_pct = round(missing_count / n * 100, 2) if n > 0 else 0.0

    if missing_count == 0:
        return {
            'missing_count': 0, 'missing_pct': 0.0,
            'missing_pattern': '无缺失', 'max_consecutive_missing': 0
        }

    # 计算最长连续缺失段
    is_null = series.isnull().astype(int).values
    max_consec = 0
    current = 0
    for v in is_null:
        if v == 1:
            current += 1
            max_consec = max(max_consec, current)
        else:
            current = 0

    # 判断缺失模式
    if max_consec >= 50 and max_consec / missing_count >= 0.3:
        pattern = '时序连续缺失'
    else:
        pattern = '随机缺失'

    return {
        'missing_count': missing_count,
        'missing_pct': missing_pct,
        'missing_pattern': pattern,
        'max_consecutive_missing': max_consec
    }


def diagnose_outliers(series: pd.Series) -> dict:
    """
    基于 IQR 法和 3-sigma 法检测单个特征的异常值。

    Description:
        1. IQR 法: Q1 - 1.5*IQR 和 Q3 + 1.5*IQR 之外为异常。
        2. 3-sigma 法: 均值 +/- 3倍标准差之外为异常。
        3. 取两种方法的并集作为最终异常值集合。
        4. 对时序数据做差分后再检测，避免趋势导致误判。

    Parameters:
        series : pd.Series
            形状为 (n_samples,) 的单列特征数据（可含 NaN）。

    Returns:
        result : dict
            - 'outlier_count_iqr': int, IQR 法检出异常值数
            - 'outlier_count_3sigma': int, 3-sigma 法检出异常值数
            - 'outlier_count_union': int, 并集异常值数
            - 'outlier_pct': float, 并集异常值占有效样本(%)
            - 'trend_adjusted_outlier_pct': float, 差分后异常值占比(%)
    """
    s = series.dropna()
    n = len(s)
    if n < 10:
        return {
            'outlier_count_iqr': 0, 'outlier_count_3sigma': 0,
            'outlier_count_union': 0, 'outlier_pct': 0.0,
            'trend_adjusted_outlier_pct': 0.0
        }

    # IQR 法
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    lower_iqr = q1 - 1.5 * iqr
    upper_iqr = q3 + 1.5 * iqr
    mask_iqr = (s < lower_iqr) | (s > upper_iqr)

    # 3-sigma 法
    mean = s.mean()
    std = s.std()
    if std > 0:
        lower_3s = mean - 3 * std
        upper_3s = mean + 3 * std
        mask_3s = (s < lower_3s) | (s > upper_3s)
    else:
        mask_3s = pd.Series(False, index=s.index)

    # 并集
    mask_union = mask_iqr | mask_3s

    # 差分后检测（去趋势）
    diff_s = s.diff().dropna()
    trend_outlier_pct = 0.0
    if len(diff_s) > 10:
        dq1 = diff_s.quantile(0.25)
        dq3 = diff_s.quantile(0.75)
        diqr = dq3 - dq1
        if diqr > 0:
            mask_diff = (diff_s < dq1 - 1.5 * diqr) | (diff_s > dq3 + 1.5 * diqr)
            trend_outlier_pct = round(mask_diff.sum() / len(diff_s) * 100, 2)

    return {
        'outlier_count_iqr': int(mask_iqr.sum()),
        'outlier_count_3sigma': int(mask_3s.sum()),
        'outlier_count_union': int(mask_union.sum()),
        'outlier_pct': round(mask_union.sum() / n * 100, 2),
        'trend_adjusted_outlier_pct': trend_outlier_pct
    }


def diagnose_distribution(series: pd.Series) -> dict:
    """
    诊断单个特征的分布特征：偏度、峰度、正态性检验和 ADF 平稳性检验。

    Description:
        1. 计算偏度(skewness)和峰度(kurtosis)。
        2. Shapiro-Wilk 正态性检验（抽样 5000 条以提高效率）。
        3. ADF 检验判断时序平稳性（抽样 10000 条）。

    Parameters:
        series : pd.Series
            形状为 (n_samples,) 的单列特征数据（可含 NaN）。

    Returns:
        result : dict
            - 'skewness': float, 偏度
            - 'kurtosis': float, 峰度
            - 'is_normal': bool, Shapiro p > 0.05 判为正态
            - 'shapiro_p': float, Shapiro-Wilk 检验 p 值
            - 'is_stationary': bool, ADF p < 0.05 判为平稳
            - 'adf_p': float, ADF 检验 p 值
    """
    s = series.dropna()
    n = len(s)

    result = {
        'skewness': 0.0, 'kurtosis': 0.0,
        'is_normal': False, 'shapiro_p': 0.0,
        'is_stationary': False, 'adf_p': 1.0
    }

    if n < 20:
        return result

    result['skewness'] = round(float(s.skew()), 4)
    result['kurtosis'] = round(float(s.kurtosis()), 4)

    # Shapiro-Wilk 正态性检验（抽样）
    try:
        sample = s.sample(min(5000, n), random_state=42)
        _, p_shapiro = sp_stats.shapiro(sample)
        result['shapiro_p'] = round(float(p_shapiro), 6)
        result['is_normal'] = p_shapiro > 0.05
    except Exception:
        pass

    # ADF 平稳性检验（抽样）
    try:
        adf_sample = s.iloc[:min(10000, n)]
        adf_result = adfuller(adf_sample, autolag='AIC', maxlag=20)
        p_adf = adf_result[1]
        result['adf_p'] = round(float(p_adf), 6)
        result['is_stationary'] = p_adf < 0.05
    except Exception:
        pass

    return result


def diagnose_correlation(series: pd.Series, df: pd.DataFrame,
                          label_cols: list) -> dict:
    """
    计算单个特征与所有标签(Y1~Y12)的 Pearson/Spearman 相关系数及显著性。

    Description:
        对每个标签列，计算:
        1. Pearson 相关系数 + p 值
        2. Spearman 秩相关系数 + p 值
        保留相关系数绝对值最大的标签及其详细指标。

    Parameters:
        series : pd.Series
            形状为 (n_samples,) 的单列特征数据（可含 NaN）。
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的完整数据，需包含 label_cols 中的列。
        label_cols : list of str
            标签列名列表，长度最多 12，如 ['Y1', ..., 'Y12']。

    Returns:
        result : dict
            - 'best_label': str, 相关性最强的标签名
            - 'best_pearson_r': float, 对应 Pearson 相关系数
            - 'best_pearson_p': float, 对应 Pearson p 值
            - 'best_spearman_r': float, 对应 Spearman 相关系数
            - 'best_spearman_p': float, 对应 Spearman p 值
            - 'all_pearson': dict, {label: r} 所有标签的 Pearson 系数
            - 'all_spearman': dict, {label: r} 所有标签的 Spearman 系数
    """
    result = {
        'best_label': '', 'best_pearson_r': 0.0, 'best_pearson_p': 1.0,
        'best_spearman_r': 0.0, 'best_spearman_p': 1.0,
        'all_pearson': {}, 'all_spearman': {}
    }

    best_abs_r = 0.0
    feat = series.dropna()

    for label in label_cols:
        if label not in df.columns:
            continue
        # 对齐非缺失索引
        common_idx = feat.index.intersection(df[label].dropna().index)
        if len(common_idx) < 30:
            continue

        x = feat.loc[common_idx].values
        y = df[label].loc[common_idx].values

        try:
            pr, pp = sp_stats.pearsonr(x, y)
        except Exception:
            pr, pp = 0.0, 1.0
        try:
            sr, sp_val = sp_stats.spearmanr(x, y)
        except Exception:
            sr, sp_val = 0.0, 1.0

        result['all_pearson'][label] = round(float(pr), 4)
        result['all_spearman'][label] = round(float(sr), 4)

        if abs(pr) > best_abs_r:
            best_abs_r = abs(pr)
            result['best_label'] = label
            result['best_pearson_r'] = round(float(pr), 4)
            result['best_pearson_p'] = round(float(pp), 6)
            result['best_spearman_r'] = round(float(sr), 4)
            result['best_spearman_p'] = round(float(sp_val), 6)

    return result


def run_feature_diagnosis(df: pd.DataFrame, feature_cols: list,
                           label_cols: list) -> list:
    """
    对所有特征执行完整诊断流程，返回结构化诊断记录列表。

    Description:
        遍历 feature_cols 中的每个特征，依次调用:
        diagnose_missing / diagnose_outliers / diagnose_distribution /
        diagnose_correlation，将结果合并为一条扁平化字典记录。
        每处理 50 个特征输出一次进度。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的完整数据。
        feature_cols : list of str
            特征列名列表，最多 300 个，如 ['X1', ..., 'X300']。
        label_cols : list of str
            标签列名列表，最多 12 个，如 ['Y1', ..., 'Y12']。

    Returns:
        diag_records : list of dict
            长度为 n_features 的列表，每条记录为一个字典，包含:
            - 'feature': str, 特征名
            - 缺失值诊断的全部字段
            - 异常值诊断的全部字段
            - 分布诊断的全部字段
            - 相关性诊断的全部字段
    """
    diag_records = []
    total = len(feature_cols)

    for i, col in enumerate(feature_cols):
        if col not in df.columns:
            continue

        series = df[col]
        record = {'feature': col}

        # 1. 缺失值诊断
        record.update(diagnose_missing(series))
        # 2. 异常值检测
        record.update(diagnose_outliers(series))
        # 3. 分布特征
        record.update(diagnose_distribution(series))
        # 4. 与标签的相关性
        record.update(diagnose_correlation(series, df, label_cols))

        diag_records.append(record)

        if (i + 1) % 50 == 0 or (i + 1) == total:
            print(f"  [PROGRESS] {i+1}/{total} 个特征诊断完成")

    return diag_records


# ---- 执行诊断 ----
print("=" * 60)
print("步骤 2.1: 逐特征诊断")
print("=" * 60 + "\n")

diag_records = run_feature_diagnosis(df, FEATURE_COLS, LABEL_COLS)
print(f"\n[OK] 诊断完成: 共 {len(diag_records)} 个特征")

步骤 2.1: 逐特征诊断

  [PROGRESS] 50/300 个特征诊断完成
  [PROGRESS] 100/300 个特征诊断完成
  [PROGRESS] 150/300 个特征诊断完成
  [PROGRESS] 200/300 个特征诊断完成
  [PROGRESS] 250/300 个特征诊断完成
  [PROGRESS] 300/300 个特征诊断完成

[OK] 诊断完成: 共 300 个特征


### 2.2 结构化特征诊断报告
将诊断结果整理为 DataFrame 表格，按 "特征名 — 缺失值 — 异常值 — 分布 — 与标签相关性" 结构化输出。

In [7]:
# =============================================================================
# Cell: 结构化特征诊断报告
# =============================================================================


def build_diagnosis_report(diag_records: list) -> pd.DataFrame:
    """
    将逐特征诊断记录列表转换为结构化报告 DataFrame。

    Description:
        1. 从 diag_records 中提取核心字段。
        2. 按缺失率降序排列。
        3. 添加综合风险等级列（基于缺失率 + 异常值占比 + 分布偏度）。
        4. 输出汇总统计信息。

    Parameters:
        diag_records : list of dict
            长度为 n_features 的诊断记录列表，每条记录来自
            run_feature_diagnosis 的输出。

    Returns:
        report_df : pd.DataFrame
            形状为 (n_features, n_report_cols) 的结构化报告表，
            包含特征名、缺失值、异常值、分布、相关性等列。
    """
    # 提取核心字段构建报告
    rows = []
    for rec in diag_records:
        rows.append({
            '特征名': rec['feature'],
            # --- 缺失值 ---
            '缺失率(%)': rec['missing_pct'],
            '缺失模式': rec['missing_pattern'],
            '最长连续缺失': rec['max_consecutive_missing'],
            # --- 异常值 ---
            '异常值数(IQR)': rec['outlier_count_iqr'],
            '异常值数(3sigma)': rec['outlier_count_3sigma'],
            '异常值占比(%)': rec['outlier_pct'],
            '去趋势异常(%)': rec['trend_adjusted_outlier_pct'],
            # --- 分布 ---
            '偏度': rec['skewness'],
            '峰度': rec['kurtosis'],
            '正态(Shapiro)': rec['is_normal'],
            'Shapiro_p': rec['shapiro_p'],
            '平稳(ADF)': rec['is_stationary'],
            'ADF_p': rec['adf_p'],
            # --- 相关性 ---
            '最强相关标签': rec['best_label'],
            '最强Pearson_r': rec['best_pearson_r'],
            '最强Pearson_p': rec['best_pearson_p'],
            '最强Spearman_r': rec['best_spearman_r'],
            '最强Spearman_p': rec['best_spearman_p'],
        })

    report_df = pd.DataFrame(rows)

    # 综合风险等级
    def risk_level(row):
        score = 0
        if row['缺失率(%)'] > 50:
            score += 3
        elif row['缺失率(%)'] > 20:
            score += 2
        elif row['缺失率(%)'] > 5:
            score += 1
        if row['异常值占比(%)'] > 10:
            score += 2
        elif row['异常值占比(%)'] > 5:
            score += 1
        if abs(row['偏度']) > 5:
            score += 1
        if score >= 4:
            return '高'
        elif score >= 2:
            return '中'
        else:
            return '低'

    report_df['风险等级'] = report_df.apply(risk_level, axis=1)

    # 排序
    report_df = report_df.sort_values('缺失率(%)', ascending=False).reset_index(drop=True)

    # ---- 汇总统计 ----
    print("=" * 60)
    print("特征诊断报告 -- 汇总统计")
    print("=" * 60)

    n = len(report_df)
    print(f"\n[COUNT] 诊断特征总数: {n}")

    # 缺失值汇总
    no_miss = (report_df['缺失率(%)'] == 0).sum()
    low_miss = ((report_df['缺失率(%)'] > 0) & (report_df['缺失率(%)'] <= 5)).sum()
    mid_miss = ((report_df['缺失率(%)'] > 5) & (report_df['缺失率(%)'] <= 30)).sum()
    high_miss = (report_df['缺失率(%)'] > 30).sum()
    consec_miss = (report_df['缺失模式'] == '时序连续缺失').sum()
    print(f"\n[MISSING] 缺失值分布:")
    print(f"  无缺失:      {no_miss}")
    print(f"  低缺失(<=5%): {low_miss}")
    print(f"  中缺失(5~30%): {mid_miss}")
    print(f"  高缺失(>30%): {high_miss}")
    print(f"  时序连续缺失: {consec_miss}")

    # 异常值汇总
    avg_outlier = report_df['异常值占比(%)'].mean()
    high_outlier = (report_df['异常值占比(%)'] > 5).sum()
    print(f"\n[OUTLIER] 异常值:")
    print(f"  平均异常值占比: {avg_outlier:.2f}%")
    print(f"  异常值占比>5%的特征: {high_outlier}")

    # 分布汇总
    normal_cnt = report_df['正态(Shapiro)'].sum()
    stationary_cnt = report_df['平稳(ADF)'].sum()
    print(f"\n[DIST] 分布特征:")
    print(f"  通过正态检验: {normal_cnt}/{n}")
    print(f"  时序平稳:     {stationary_cnt}/{n}")
    print(f"  平均偏度: {report_df['偏度'].mean():.4f}, 平均峰度: {report_df['峰度'].mean():.4f}")

    # 相关性汇总
    strong_corr = (report_df['最强Pearson_r'].abs() > 0.3).sum()
    sig_corr = (report_df['最强Pearson_p'] < 0.05).sum()
    print(f"\n[CORR] 与标签相关性:")
    print(f"  |Pearson r| > 0.3: {strong_corr}")
    print(f"  Pearson p < 0.05 (显著): {sig_corr}")

    # 风险等级汇总
    risk_vc = report_df['风险等级'].value_counts()
    print(f"\n[RISK] 综合风险等级:")
    for level in ['低', '中', '高']:
        print(f"  {level}: {risk_vc.get(level, 0)}")

    print("=" * 60)

    # 展示部分结果
    display_cols = ['特征名', '缺失率(%)', '缺失模式', '异常值占比(%)',
                    '偏度', '峰度', '平稳(ADF)', '最强相关标签',
                    '最强Pearson_r', '风险等级']
    print("\n[TABLE] 诊断报告 Top20 (按缺失率降序):")
    print(report_df[display_cols].head(20).to_string(index=False))

    return report_df


# 执行
diag_df = build_diagnosis_report(diag_records)
diag_df.to_csv('feature_diagnosis_report.csv', index=False, encoding='utf-8-sig')
print(f"\n[OK] 诊断报告已保存: feature_diagnosis_report.csv")

特征诊断报告 -- 汇总统计

[COUNT] 诊断特征总数: 300

[MISSING] 缺失值分布:
  无缺失:      0
  低缺失(<=5%): 0
  中缺失(5~30%): 300
  高缺失(>30%): 0
  时序连续缺失: 0

[OUTLIER] 异常值:
  平均异常值占比: 6.37%
  异常值占比>5%的特征: 129

[DIST] 分布特征:
  通过正态检验: 10/300
  时序平稳:     300/300
  平均偏度: 3.8438, 平均峰度: 1512.9559

[CORR] 与标签相关性:
  |Pearson r| > 0.3: 0
  Pearson p < 0.05 (显著): 282

[RISK] 综合风险等级:
  低: 0
  中: 200
  高: 100

[TABLE] 诊断报告 Top20 (按缺失率降序):
 特征名  缺失率(%) 缺失模式  异常值占比(%)       偏度         峰度  平稳(ADF) 最强相关标签  最强Pearson_r 风险等级
X170 24.6800 随机缺失    1.3400 113.2731 13910.7393     True    Y12      -0.0102    中
X200 24.6600 随机缺失    5.6100   0.1020    24.7083     True    Y12       0.0072    中
X199 24.6600 随机缺失    3.3500   0.6048     1.3712     True     Y5       0.1260    中
X198 24.6600 随机缺失    3.2800   0.6769     1.5248     True     Y5       0.1349    中
X197 24.6600 随机缺失    3.2100   0.8050     1.8843     True     Y1       0.1475    中
X196 24.6600 随机缺失    4.4000   2.4737    15.8261     True     Y1       0.1323    中
X195 24.6600 随机缺失    4.0

### 2.3 诊断结果可视化
绘制三组关键图表：
1. **缺失值排名 Top20 柱状图** — 展示缺失率最高的 20 个特征；
2. **异常值分布箱型图** — 选取异常值占比最高的 20 个特征绘制箱线图；
3. **特征-标签相关性热力图** — 展示所有特征与 Y1~Y12 的 Pearson 相关系数矩阵。

In [8]:
# =============================================================================
# Cell: 诊断结果可视化
# =============================================================================


def plot_missing_top20(diag_df: pd.DataFrame, save_dir: str = IMG_DIR) -> None:
    """
    绘制缺失率排名 Top20 的特征柱状图。

    Description:
        按缺失率降序取前 20 个特征，绘制水平柱状图，
        并用颜色区分缺失模式（随机/时序连续）。

    Parameters:
        diag_df : pd.DataFrame
            形状为 (n_features, n_report_cols) 的诊断报告表。
        save_dir : str
            图片保存目录路径，标量字符串。

    Returns:
        None
    """
    top20 = diag_df.nlargest(20, '缺失率(%)').sort_values('缺失率(%)')

    fig, ax = plt.subplots(figsize=(10, 7))
    colors = ['#F44336' if p == '时序连续缺失' else '#2196F3'
              for p in top20['缺失模式']]
    bars = ax.barh(top20['特征名'], top20['缺失率(%)'], color=colors, edgecolor='black', linewidth=0.3)

    # 标注数值
    for bar, pct in zip(bars, top20['缺失率(%)']):
        ax.text(bar.get_width() + 0.3, bar.get_y() + bar.get_height() / 2,
                f'{pct:.1f}%', va='center', fontsize=8)

    ax.set_xlabel('缺失率 (%)')
    ax.set_title('特征缺失率 Top20', fontsize=13)
    ax.grid(True, alpha=0.3, axis='x')

    # 图例
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='#F44336', label='时序连续缺失'),
                       Patch(facecolor='#2196F3', label='随机缺失')]
    ax.legend(handles=legend_elements, loc='lower right')

    plt.tight_layout()
    save_path = os.path.join(save_dir, 'missing_top20.png')
    fig.savefig(save_path, bbox_inches='tight')
    plt.show()
    print(f"[OK] 缺失率 Top20 图已保存: {save_path}")


def plot_outlier_boxplots(df: pd.DataFrame, diag_df: pd.DataFrame,
                           save_dir: str = IMG_DIR) -> None:
    """
    绘制异常值占比最高的 Top20 特征的箱型图。

    Description:
        按异常值占比降序取前 20 个特征，绘制箱型图以展示数据分布和离群点。
        对数据做标准化处理后绘图，使不同量纲的特征可以对比。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据。
        diag_df : pd.DataFrame
            形状为 (n_features, n_report_cols) 的诊断报告表。
        save_dir : str
            图片保存目录路径，标量字符串。

    Returns:
        None
    """
    top20 = diag_df.nlargest(20, '异常值占比(%)')
    top20_cols = top20['特征名'].tolist()

    # 标准化后绘图
    plot_data = df[top20_cols].copy()
    for c in top20_cols:
        s = plot_data[c]
        s_mean = s.mean()
        s_std = s.std()
        if s_std > 0:
            plot_data[c] = (s - s_mean) / s_std

    fig, ax = plt.subplots(figsize=(16, 6))
    plot_data[top20_cols].boxplot(ax=ax, vert=True, showfliers=True,
                                  flierprops=dict(marker='o', markersize=2, alpha=0.3))
    ax.set_title('异常值占比 Top20 特征箱型图（标准化后）', fontsize=13)
    ax.set_xlabel('特征')
    ax.set_ylabel('标准化值')
    ax.tick_params(axis='x', rotation=45, labelsize=8)
    ax.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    save_path = os.path.join(save_dir, 'outlier_boxplot_top20.png')
    fig.savefig(save_path, bbox_inches='tight')
    plt.show()
    print(f"[OK] 异常值箱型图已保存: {save_path}")


def plot_correlation_heatmap(diag_records: list, label_cols: list,
                              save_dir: str = IMG_DIR) -> None:
    """
    绘制特征与标签(Y1~Y12)的 Pearson 相关系数热力图。

    Description:
        1. 从 diag_records 中提取每个特征与所有标签的 Pearson 相关系数。
        2. 构建 (n_features, n_labels) 的相关系数矩阵。
        3. 使用 seaborn heatmap 绘制，颜色映射从蓝(-1)到红(+1)。
        4. 为保持可读性，y轴仅标注每10个特征。

    Parameters:
        diag_records : list of dict
            长度为 n_features 的诊断记录列表。
        label_cols : list of str
            标签列名列表，长度最多 12。
        save_dir : str
            图片保存目录路径，标量字符串。

    Returns:
        None
    """
    # 构建相关系数矩阵
    corr_data = {}
    features = []
    for rec in diag_records:
        feat = rec['feature']
        features.append(feat)
        for label in label_cols:
            if label not in corr_data:
                corr_data[label] = []
            corr_data[label].append(rec['all_pearson'].get(label, 0.0))

    corr_matrix = pd.DataFrame(corr_data, index=features)

    fig, ax = plt.subplots(figsize=(14, 18))
    sns.heatmap(corr_matrix, cmap='RdBu_r', center=0, vmin=-0.5, vmax=0.5,
                xticklabels=True, yticklabels=10,
                cbar_kws={'label': 'Pearson r', 'shrink': 0.6},
                ax=ax, linewidths=0)
    ax.set_title('特征 X1~X300 与标签 Y1~Y12 Pearson 相关系数热力图', fontsize=13)
    ax.set_xlabel('标签')
    ax.set_ylabel('特征')
    ax.tick_params(axis='y', labelsize=6)

    plt.tight_layout()
    save_path = os.path.join(save_dir, 'feature_label_corr_heatmap.png')
    fig.savefig(save_path, bbox_inches='tight')
    plt.show()
    print(f"[OK] 相关性热力图已保存: {save_path}")


# ---- 执行可视化 ----
print("=" * 60)
print("步骤 2.3: 诊断结果可视化")
print("=" * 60)

print("\n[1/3] 缺失值排名 Top20")
plot_missing_top20(diag_df)

print("\n[2/3] 异常值分布箱型图 Top20")
plot_outlier_boxplots(df, diag_df)

print("\n[3/3] 特征-标签相关性热力图")
plot_correlation_heatmap(diag_records, LABEL_COLS)

步骤 2.3: 诊断结果可视化

[1/3] 缺失值排名 Top20
[OK] 缺失率 Top20 图已保存: images\missing_top20.png

[2/3] 异常值分布箱型图 Top20
[OK] 异常值箱型图已保存: images\outlier_boxplot_top20.png

[3/3] 特征-标签相关性热力图
[OK] 相关性热力图已保存: images\feature_label_corr_heatmap.png


## 步骤3：自动特征清理

基于步骤2诊断结果，利用千问大模型 Agent **自主决策**每个特征的清理方案，然后自动执行清理：
- **缺失值处理**：时序连续缺失 -> 线性插值/样条插值；随机缺失 -> 均值/中位数填充；缺失>80% -> 标记剔除
- **异常值处理**：温和异常 -> 上下限截断(Winsorize)；极端异常 -> 中位数替换；时序趋势性异常 -> 保留标注
- **分布优化**：非平稳特征 -> 差分/标准化处理，验证处理后平稳性

### 3.1 千问 Agent 决策清理方案
将诊断报告分批发送给千问大模型，由 Agent 针对每个特征的诊断结果（缺失模式、异常值、分布、相关性）自主输出结构化清理方案。

In [9]:
# =============================================================================
# Cell: 千问 Agent 决策清理方案
# =============================================================================
from openai import OpenAI
import json, re, time

# API 配置
QWEN_API_KEY = "sk-pxizupebbwgijptggmseledfboqcfcqcjltbfiucswhxicow"
QWEN_BASE_URL = "https://api.siliconflow.cn/v1"
QWEN_MODEL = "Qwen/Qwen2.5-7B-Instruct"


def call_qwen_agent(prompt: str, system_prompt: str = None) -> str:
    """
    调用千问 7B API (硅基流动接口)，获取 Agent 决策结果。

    Description:
        通过 OpenAI 兼容接口连接硅基流动平台，向 Qwen2.5-7B-Instruct 模型
        发送 system + user 消息，流式接收并拼接完整回复文本。
        若调用失败则返回 None，由上层函数执行 fallback 逻辑。

    Parameters:
        prompt : str
            用户 prompt 文本，标量字符串。
        system_prompt : str, optional
            系统 prompt 文本，标量字符串，默认为 None。

    Returns:
        full_response : str or None
            模型的完整回复文本，标量字符串。
            若 API 调用失败则返回 None。
    """
    try:
        client = OpenAI(api_key=QWEN_API_KEY, base_url=QWEN_BASE_URL)
        messages = []
        if system_prompt:
            messages.append({'role': 'system', 'content': system_prompt})
        messages.append({'role': 'user', 'content': prompt})

        response = client.chat.completions.create(
            model=QWEN_MODEL, messages=messages, stream=True
        )
        full_response = ""
        for chunk in response:
            if not chunk.choices:
                continue
            delta = chunk.choices[0].delta
            if delta.content:
                full_response += delta.content
        return full_response
    except Exception as e:
        logger.error(f"[ERROR] 千问 API 调用失败: {e}")
        return None


def build_cleaning_prompt_batch(diag_df: pd.DataFrame,
                                  batch_features: list) -> str:
    """
    为一批特征构建清理方案决策 prompt。

    Description:
        将每个特征的诊断结果（缺失率、缺失模式、异常值占比、偏度、峰度、
        平稳性、去趋势异常率）格式化为结构化文本，发送给 Agent。

    Parameters:
        diag_df : pd.DataFrame
            形状为 (n_features, n_report_cols) 的诊断报告表。
        batch_features : list of str
            本批次的特征名列表，长度一般为 30。

    Returns:
        prompt : str
            组装好的 user prompt 文本。
    """
    lines = []
    for feat in batch_features:
        row = diag_df[diag_df['特征名'] == feat]
        if row.empty:
            continue
        r = row.iloc[0]
        lines.append(
            f"- {feat}: 缺失率={r['缺失率(%)']}%, 缺失模式={r['缺失模式']}, "
            f"最长连续缺失={r['最长连续缺失']}, "
            f"异常值占比={r['异常值占比(%)']}%, 去趋势异常={r['去趋势异常(%)']}%, "
            f"偏度={r['偏度']}, 峰度={r['峰度']}, "
            f"平稳(ADF)={r['平稳(ADF)']}, ADF_p={r['ADF_p']}"
        )
    feature_text = "\n".join(lines)

    prompt = f"""以下是 {len(batch_features)} 个金融时序特征的诊断结果：

{feature_text}

请为每个特征输出清理方案，严格按照以下JSON数组格式回复，不要添加其他内容：
[
  {{
    "feature": "X1",
    "missing_action": "linear_interp|spline_interp|mean_fill|median_fill|drop",
    "missing_reason": "理由",
    "outlier_action": "winsorize|median_replace|keep",
    "outlier_reason": "理由",
    "distribution_action": "diff|standardize|normalize|none",
    "distribution_reason": "理由"
  }}
]

决策规则参考：
1. 缺失值: 时序连续缺失用linear_interp或spline_interp; 随机缺失用mean_fill或median_fill; 缺失率>80%用drop
2. 异常值: 去趋势异常率<异常值占比(说明异常源于趋势)用keep; 否则异常值占比>5%用winsorize; 极端情况用median_replace
3. 分布: 非平稳(ADF=False)且偏度绝对值>2用diff; 非平稳但偏度不大用standardize; 平稳的用none"""

    return prompt


def parse_agent_cleaning_response(response: str,
                                    batch_features: list) -> list:
    """
    解析 Agent 返回的 JSON 清理方案，若解析失败则启用规则引擎 fallback。

    Description:
        1. 尝试从 response 中提取 JSON 数组。
        2. 若提取失败，对每个特征用规则引擎生成默认方案。

    Parameters:
        response : str or None
            Agent 的回复文本，应包含 JSON 数组。
        batch_features : list of str
            本批次的特征名列表。

    Returns:
        plans : list of dict
            长度为 len(batch_features) 的清理方案列表。
    """
    plans = []

    if response:
        try:
            # 提取 JSON 数组
            match = re.search(r'\[.*\]', response, re.DOTALL)
            if match:
                parsed = json.loads(match.group())
                if isinstance(parsed, list) and len(parsed) > 0:
                    plans = parsed
        except (json.JSONDecodeError, Exception):
            pass

    # 检查是否完整覆盖
    parsed_features = {p.get('feature', '') for p in plans}
    missing_feats = [f for f in batch_features if f not in parsed_features]

    return plans, missing_feats


def rule_engine_fallback(diag_df: pd.DataFrame, feature: str) -> dict:
    """
    规则引擎 fallback：当 Agent 未返回有效方案时，基于规则自动决策。

    Description:
        根据诊断报告中的缺失率、缺失模式、异常值占比、去趋势异常率、
        ADF平稳性和偏度，生成对应的清理方案。

    Parameters:
        diag_df : pd.DataFrame
            形状为 (n_features, n_report_cols) 的诊断报告表。
        feature : str
            特征名，标量字符串。

    Returns:
        plan : dict
            包含 feature, missing_action, outlier_action, distribution_action
            及对应 reason 的清理方案字典。
    """
    row = diag_df[diag_df['特征名'] == feature]
    if row.empty:
        return {'feature': feature,
                'missing_action': 'median_fill', 'missing_reason': '默认',
                'outlier_action': 'winsorize', 'outlier_reason': '默认',
                'distribution_action': 'none', 'distribution_reason': '默认'}

    r = row.iloc[0]
    plan = {'feature': feature}

    # 缺失值决策
    if r['缺失率(%)'] > 80:
        plan['missing_action'] = 'drop'
        plan['missing_reason'] = f"缺失率{r['缺失率(%)']}%过高，标记剔除"
    elif r['缺失模式'] == '时序连续缺失':
        plan['missing_action'] = 'linear_interp'
        plan['missing_reason'] = f"时序连续缺失(最长{r['最长连续缺失']}),用线性插值"
    elif r['缺失率(%)'] > 0:
        plan['missing_action'] = 'median_fill'
        plan['missing_reason'] = f"随机缺失{r['缺失率(%)']}%,用中位数填充"
    else:
        plan['missing_action'] = 'none'
        plan['missing_reason'] = '无缺失'

    # 异常值决策
    if r['去趋势异常(%)'] < r['异常值占比(%)'] * 0.5:
        plan['outlier_action'] = 'keep'
        plan['outlier_reason'] = '异常主要源于趋势,保留'
    elif r['异常值占比(%)'] > 10:
        plan['outlier_action'] = 'median_replace'
        plan['outlier_reason'] = f"极端异常{r['异常值占比(%)']}%,中位数替换"
    elif r['异常值占比(%)'] > 2:
        plan['outlier_action'] = 'winsorize'
        plan['outlier_reason'] = f"温和异常{r['异常值占比(%)']}%,截断处理"
    else:
        plan['outlier_action'] = 'keep'
        plan['outlier_reason'] = '异常值较少,保留'

    # 分布优化决策
    if not r['平稳(ADF)'] and abs(r['偏度']) > 2:
        plan['distribution_action'] = 'diff'
        plan['distribution_reason'] = f"非平稳且偏度={r['偏度']},做差分"
    elif not r['平稳(ADF)']:
        plan['distribution_action'] = 'standardize'
        plan['distribution_reason'] = '非平稳,标准化处理'
    else:
        plan['distribution_action'] = 'none'
        plan['distribution_reason'] = '已平稳,无需处理'

    return plan


def get_all_cleaning_plans(diag_df: pd.DataFrame,
                            feature_cols: list,
                            batch_size: int = 30) -> pd.DataFrame:
    """
    分批调用千问 Agent 获取所有特征的清理方案，不足部分用规则引擎补全。

    Description:
        1. 将 300 个特征分为 batch_size 大小的批次。
        2. 每批构建 prompt 发送给 Agent，获取清理方案。
        3. Agent 未覆盖的特征用 rule_engine_fallback 补全。
        4. 汇总所有方案为 DataFrame。

    Parameters:
        diag_df : pd.DataFrame
            形状为 (n_features, n_report_cols) 的诊断报告表。
        feature_cols : list of str
            特征列名列表，长度最多 300。
        batch_size : int
            每批发送给 Agent 的特征数，标量整数，默认 30。

    Returns:
        plans_df : pd.DataFrame
            形状为 (n_features, 7) 的清理方案表，包含:
            feature, missing_action, missing_reason, outlier_action,
            outlier_reason, distribution_action, distribution_reason
    """
    system_prompt = """你是一个专业的金融时序数据清理Agent。
你需要根据每个特征的诊断结果，为其制定最优的数据清理方案。
请严格按照JSON数组格式回复。不要回复JSON以外的内容。"""

    all_plans = []
    features_in_diag = diag_df['特征名'].tolist()
    cols_to_process = [c for c in feature_cols if c in features_in_diag]

    n_batches = (len(cols_to_process) + batch_size - 1) // batch_size

    for b in range(n_batches):
        start = b * batch_size
        end = min(start + batch_size, len(cols_to_process))
        batch = cols_to_process[start:end]

        print(f"  [BATCH {b+1}/{n_batches}] 正在请求 Agent 决策 ({len(batch)} 个特征)...")

        prompt = build_cleaning_prompt_batch(diag_df, batch)
        response = call_qwen_agent(prompt, system_prompt)

        agent_plans, missing_feats = parse_agent_cleaning_response(response, batch)

        if agent_plans:
            all_plans.extend(agent_plans)
            print(f"    Agent 返回 {len(agent_plans)} 个方案", end="")
        else:
            missing_feats = batch
            print(f"    Agent 未返回有效方案", end="")

        # 用规则引擎补全
        if missing_feats:
            for feat in missing_feats:
                fb = rule_engine_fallback(diag_df, feat)
                all_plans.append(fb)
            print(f", 规则引擎补全 {len(missing_feats)} 个")
        else:
            print()

        # 避免请求过快
        if b < n_batches - 1:
            time.sleep(1)

    plans_df = pd.DataFrame(all_plans)

    # 确保列名统一
    for col in ['feature', 'missing_action', 'missing_reason',
                'outlier_action', 'outlier_reason',
                'distribution_action', 'distribution_reason']:
        if col not in plans_df.columns:
            plans_df[col] = ''

    plans_df = plans_df[['feature', 'missing_action', 'missing_reason',
                          'outlier_action', 'outlier_reason',
                          'distribution_action', 'distribution_reason']]

    return plans_df


# ---- 执行 Agent 决策 ----
print("=" * 60)
print("步骤 3.1: 千问 Agent 决策清理方案")
print("=" * 60 + "\n")

cleaning_plans = get_all_cleaning_plans(diag_df, FEATURE_COLS, batch_size=30)

print(f"\n[OK] 共获取 {len(cleaning_plans)} 个特征清理方案")

# 统计方案分布
print(f"\n[SUMMARY] 清理方案分布:")
print(f"  缺失值处理:")
for action, cnt in cleaning_plans['missing_action'].value_counts().items():
    print(f"    {action}: {cnt}")
print(f"  异常值处理:")
for action, cnt in cleaning_plans['outlier_action'].value_counts().items():
    print(f"    {action}: {cnt}")
print(f"  分布处理:")
for action, cnt in cleaning_plans['distribution_action'].value_counts().items():
    print(f"    {action}: {cnt}")

# 标记待剔除特征
drop_features = cleaning_plans[cleaning_plans['missing_action'] == 'drop']['feature'].tolist()
print(f"\n  待剔除特征 (缺失>80%): {len(drop_features)} 个")
if drop_features:
    print(f"    {drop_features}")

# 保存方案
cleaning_plans.to_csv('cleaning_plans.csv', index=False, encoding='utf-8-sig')
print(f"\n[OK] 清理方案已保存: cleaning_plans.csv")

步骤 3.1: 千问 Agent 决策清理方案

  [BATCH 1/10] 正在请求 Agent 决策 (30 个特征)...


[2026-02-28 17:20:03,994] INFO: HTTP Request: POST https://api.siliconflow.cn/v1/chat/completions "HTTP/1.1 200 OK"


    Agent 未返回有效方案, 规则引擎补全 30 个
  [BATCH 2/10] 正在请求 Agent 决策 (30 个特征)...


[2026-02-28 17:20:11,976] INFO: HTTP Request: POST https://api.siliconflow.cn/v1/chat/completions "HTTP/1.1 200 OK"


    Agent 返回 30 个方案
  [BATCH 3/10] 正在请求 Agent 决策 (30 个特征)...


...

    Agent 返回 30 个方案
  [BATCH 9/10] 正在请求 Agent 决策 (30 个特征)...


[2026-02-28 17:24:20,190] INFO: HTTP Request: POST https://api.siliconflow.cn/v1/chat/completions "HTTP/1.1 200 OK"


    Agent 返回 30 个方案
  [BATCH 10/10] 正在请求 Agent 决策 (30 个特征)...


[2026-02-28 17:24:50,208] INFO: HTTP Request: POST https://api.siliconflow.cn/v1/chat/completions "HTTP/1.1 200 OK"


    Agent 返回 30 个方案

[OK] 共获取 300 个特征清理方案

[SUMMARY] 清理方案分布:
  缺失值处理:
    median_fill: 91
    mean_fill: 52
    spline_interp|mean_fill: 30
    mean_fill|median_fill: 30
    spline_interp|mean_fill|median_fill: 30
    spline_interp|median_fill: 30
    linear_interp|spline_interp|mean_fill: 21
    linear_interp|spline_interp: 12
    linear_interp: 4
  异常值处理:
    winsorize: 128
    keep: 123
    median_replace: 49
  分布处理:
    standardize: 171
    none: 85
    diff: 44

  待剔除特征 (缺失>80%): 0 个

[OK] 清理方案已保存: cleaning_plans.csv


### 3.2 执行特征清理
根据 Agent 决策的清理方案，封装 `clean_features` 函数执行清理，并输出清理前后的特征对比报告。

In [10]:
# =============================================================================
# Cell: 执行特征清理 — clean_features 函数
# =============================================================================
from statsmodels.tsa.stattools import adfuller as adf_test


def clean_features(df: pd.DataFrame, plans_df: pd.DataFrame,
                    feature_cols: list) -> tuple:
    """
    根据 Agent 决策的清理方案，对所有特征执行清理操作。

    Description:
        对每个特征依次执行三步清理:
        1. 缺失值处理: linear_interp / spline_interp / mean_fill /
           median_fill / drop (标记) / none
        2. 异常值处理: winsorize(IQR截断) / median_replace / keep
        3. 分布优化: diff(一阶差分) / standardize(z-score) /
           normalize(min-max) / none
        记录每步清理的变化量，生成清理日志。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据。
        plans_df : pd.DataFrame
            形状为 (n_features, 7) 的清理方案表，由 Agent 生成。
        feature_cols : list of str
            特征列名列表，长度最多 300。

    Returns:
        df_cleaned : pd.DataFrame
            形状为 (n_samples, n_columns) 的清理后数据。
        clean_log : list of dict
            长度为 n_features 的清理日志列表，每条记录包含:
            - 'feature': str
            - 'action_missing': str, 执行的缺失值处理
            - 'action_outlier': str, 执行的异常值处理
            - 'action_dist': str, 执行的分布处理
            - 'missing_before/after': int
            - 'outlier_before/after': int
            - 'dropped': bool
    """
    df_cleaned = df.copy()
    clean_log = []
    dropped_features = []

    plan_dict = {}
    for _, row in plans_df.iterrows():
        plan_dict[row['feature']] = row

    total = len(feature_cols)
    for i, col in enumerate(feature_cols):
        if col not in df_cleaned.columns:
            continue

        plan = plan_dict.get(col, None)
        if plan is None:
            continue

        log_entry = {
            'feature': col,
            'action_missing': str(plan.get('missing_action', 'none')),
            'action_outlier': str(plan.get('outlier_action', 'keep')),
            'action_dist': str(plan.get('distribution_action', 'none')),
            'dropped': False
        }

        s = df_cleaned[col]

        # ---- 记录清理前状态 ----
        missing_before = int(s.isnull().sum())
        # IQR 异常值计数
        s_clean = s.dropna()
        if len(s_clean) > 10:
            q1, q3 = s_clean.quantile(0.25), s_clean.quantile(0.75)
            iqr = q3 - q1
            outlier_before = int(((s_clean < q1 - 1.5 * iqr) |
                                   (s_clean > q3 + 1.5 * iqr)).sum())
        else:
            outlier_before = 0
            q1, q3, iqr = 0, 0, 0

        log_entry['missing_before'] = missing_before
        log_entry['outlier_before'] = outlier_before

        # ---- 1. 缺失值处理 ----
        m_action = str(plan.get('missing_action', 'none')).lower().strip()
        if m_action == 'drop':
            dropped_features.append(col)
            log_entry['dropped'] = True
            log_entry['missing_after'] = missing_before
            log_entry['outlier_after'] = outlier_before
            clean_log.append(log_entry)
            continue
        elif m_action == 'linear_interp':
            df_cleaned[col] = df_cleaned[col].interpolate(method='linear')
            # 头尾仍有 NaN 时用前/后填充收尾
            df_cleaned[col] = df_cleaned[col].ffill().bfill()
        elif m_action == 'spline_interp':
            try:
                df_cleaned[col] = df_cleaned[col].interpolate(method='spline', order=3)
            except Exception:
                df_cleaned[col] = df_cleaned[col].interpolate(method='linear')
            df_cleaned[col] = df_cleaned[col].ffill().bfill()
        elif m_action == 'mean_fill':
            df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].mean())
        elif m_action == 'median_fill':
            df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].median())
        # else: none — 不做处理

        # ---- 2. 异常值处理 ----
        o_action = str(plan.get('outlier_action', 'keep')).lower().strip()
        s2 = df_cleaned[col].dropna()
        if len(s2) > 10 and o_action != 'keep':
            q1_new = s2.quantile(0.25)
            q3_new = s2.quantile(0.75)
            iqr_new = q3_new - q1_new
            lower = q1_new - 1.5 * iqr_new
            upper = q3_new + 1.5 * iqr_new

            if o_action == 'winsorize':
                # 截断到上下限
                df_cleaned[col] = df_cleaned[col].clip(lower=lower, upper=upper)
            elif o_action == 'median_replace':
                # 极端异常用中位数替换
                median_val = s2.median()
                mask = (df_cleaned[col] < lower) | (df_cleaned[col] > upper)
                df_cleaned.loc[mask, col] = median_val

        # ---- 3. 分布优化 ----
        d_action = str(plan.get('distribution_action', 'none')).lower().strip()
        if d_action == 'diff':
            # 一阶差分（第一个值填0，保持长度）
            original = df_cleaned[col].copy()
            df_cleaned[col] = df_cleaned[col].diff()
            df_cleaned.loc[df_cleaned.index[0], col] = 0
            # 差分后的 NaN 用0填充
            df_cleaned[col] = df_cleaned[col].fillna(0)
        elif d_action == 'standardize':
            mean_val = df_cleaned[col].mean()
            std_val = df_cleaned[col].std()
            if std_val > 0:
                df_cleaned[col] = (df_cleaned[col] - mean_val) / std_val
        elif d_action == 'normalize':
            min_val = df_cleaned[col].min()
            max_val = df_cleaned[col].max()
            if max_val > min_val:
                df_cleaned[col] = (df_cleaned[col] - min_val) / (max_val - min_val)
        # else: none

        # ---- 记录清理后状态 ----
        missing_after = int(df_cleaned[col].isnull().sum())
        s3 = df_cleaned[col].dropna()
        if len(s3) > 10:
            q1_f = s3.quantile(0.25)
            q3_f = s3.quantile(0.75)
            iqr_f = q3_f - q1_f
            outlier_after = int(((s3 < q1_f - 1.5 * iqr_f) |
                                  (s3 > q3_f + 1.5 * iqr_f)).sum())
        else:
            outlier_after = 0

        log_entry['missing_after'] = missing_after
        log_entry['outlier_after'] = outlier_after
        clean_log.append(log_entry)

        if (i + 1) % 50 == 0 or (i + 1) == total:
            print(f"  [PROGRESS] {i+1}/{total} 个特征清理完成")

    # 输出待剔除特征
    if dropped_features:
        print(f"\n  [DROP] 标记剔除 {len(dropped_features)} 个特征: {dropped_features}")

    return df_cleaned, clean_log


# ---- 执行清理 ----
print("=" * 60)
print("步骤 3.2: 执行特征清理")
print("=" * 60 + "\n")

df_cleaned, clean_log = clean_features(df, cleaning_plans, FEATURE_COLS)

# 记录被剔除的特征
dropped_features = [e['feature'] for e in clean_log if e.get('dropped', False)]
active_features = [c for c in FEATURE_COLS if c not in dropped_features and c in df_cleaned.columns]

print(f"\n[OK] 清理完成")
print(f"  总特征数: {len(FEATURE_COLS)}")
print(f"  有效特征数: {len(active_features)}")
print(f"  剔除特征数: {len(dropped_features)}")

步骤 3.2: 执行特征清理

  [PROGRESS] 50/300 个特征清理完成
  [PROGRESS] 100/300 个特征清理完成
  [PROGRESS] 150/300 个特征清理完成
  [PROGRESS] 200/300 个特征清理完成
  [PROGRESS] 250/300 个特征清理完成
  [PROGRESS] 300/300 个特征清理完成

[OK] 清理完成
  总特征数: 300
  有效特征数: 300
  剔除特征数: 0


### 3.3 清理前后对比报告
输出清理前后的缺失值、异常值变化对比，并可视化关键变化指标。

In [11]:
# =============================================================================
# Cell: 清理前后对比报告与可视化
# =============================================================================


def generate_cleaning_comparison(clean_log: list, save_dir: str = IMG_DIR) -> pd.DataFrame:
    """
    生成清理前后对比报告，并可视化关键变化指标。

    Description:
        1. 将 clean_log 转为 DataFrame，计算缺失值/异常值的变化量和变化率。
        2. 输出汇总统计（总缺失值减少量、总异常值减少量等）。
        3. 绘制缺失值变化对比图和异常值变化对比图。

    Parameters:
        clean_log : list of dict
            长度为 n_features 的清理日志列表，
            由 clean_features 函数返回。
        save_dir : str
            图片保存目录路径，标量字符串，默认为 IMG_DIR。

    Returns:
        comparison_df : pd.DataFrame
            形状为 (n_features, n_cols) 的对比报告表。
    """
    comp_df = pd.DataFrame(clean_log)

    # 计算变化量
    comp_df['missing_delta'] = comp_df['missing_before'] - comp_df['missing_after']
    comp_df['outlier_delta'] = comp_df['outlier_before'] - comp_df['outlier_after']

    # 变化率
    comp_df['missing_change_pct'] = np.where(
        comp_df['missing_before'] > 0,
        (comp_df['missing_delta'] / comp_df['missing_before'] * 100).round(1),
        0.0
    )
    comp_df['outlier_change_pct'] = np.where(
        comp_df['outlier_before'] > 0,
        (comp_df['outlier_delta'] / comp_df['outlier_before'] * 100).round(1),
        0.0
    )

    # ---- 汇总统计 ----
    active = comp_df[~comp_df['dropped']]
    total_missing_before = int(active['missing_before'].sum())
    total_missing_after = int(active['missing_after'].sum())
    total_outlier_before = int(active['outlier_before'].sum())
    total_outlier_after = int(active['outlier_after'].sum())

    print("=" * 60)
    print("清理前后对比报告")
    print("=" * 60)
    print(f"\n[MISSING] 缺失值变化 (有效特征):")
    print(f"  清理前总缺失值: {total_missing_before:,}")
    print(f"  清理后总缺失值: {total_missing_after:,}")
    print(f"  减少量: {total_missing_before - total_missing_after:,} "
          f"({(total_missing_before - total_missing_after) / max(total_missing_before, 1) * 100:.1f}%)")

    print(f"\n[OUTLIER] 异常值变化 (有效特征):")
    print(f"  清理前总异常值: {total_outlier_before:,}")
    print(f"  清理后总异常值: {total_outlier_after:,}")
    print(f"  减少量: {total_outlier_before - total_outlier_after:,} "
          f"({(total_outlier_before - total_outlier_after) / max(total_outlier_before, 1) * 100:.1f}%)")

    # 各清理动作统计
    print(f"\n[ACTIONS] 清理动作统计:")
    print(f"  缺失值处理: {active['action_missing'].value_counts().to_dict()}")
    print(f"  异常值处理: {active['action_outlier'].value_counts().to_dict()}")
    print(f"  分布优化:   {active['action_dist'].value_counts().to_dict()}")

    # 仍有缺失值的特征
    still_missing = active[active['missing_after'] > 0]
    print(f"\n  清理后仍有缺失的特征: {len(still_missing)}")
    if len(still_missing) > 0:
        for _, r in still_missing.head(10).iterrows():
            print(f"    {r['feature']}: {r['missing_after']} 个缺失")

    # ---- 可视化 ----
    # 1) 缺失值变化前后对比 (Top20 变化量最大的)
    top_missing = active.nlargest(20, 'missing_delta').sort_values('missing_delta')
    if len(top_missing) > 0 and top_missing['missing_delta'].sum() > 0:
        fig, ax = plt.subplots(figsize=(10, 6))
        y_pos = range(len(top_missing))
        ax.barh(y_pos, top_missing['missing_before'], height=0.4, color='#F44336',
                alpha=0.7, label='清理前')
        ax.barh([y + 0.4 for y in y_pos], top_missing['missing_after'], height=0.4,
                color='#4CAF50', alpha=0.7, label='清理后')
        ax.set_yticks([y + 0.2 for y in y_pos])
        ax.set_yticklabels(top_missing['feature'], fontsize=8)
        ax.set_xlabel('缺失值数量')
        ax.set_title('缺失值变化 Top20 (清理前 vs 清理后)', fontsize=13)
        ax.legend()
        ax.grid(True, alpha=0.3, axis='x')
        plt.tight_layout()
        save_path = os.path.join(save_dir, 'missing_before_after.png')
        fig.savefig(save_path, bbox_inches='tight')
        plt.show()
        print(f"[OK] 缺失值变化图已保存: {save_path}")

    # 2) 异常值变化前后对比 (Top20 变化量最大的)
    top_outlier = active.nlargest(20, 'outlier_delta').sort_values('outlier_delta')
    if len(top_outlier) > 0 and top_outlier['outlier_delta'].sum() > 0:
        fig, ax = plt.subplots(figsize=(10, 6))
        y_pos = range(len(top_outlier))
        ax.barh(y_pos, top_outlier['outlier_before'], height=0.4, color='#FF9800',
                alpha=0.7, label='清理前')
        ax.barh([y + 0.4 for y in y_pos], top_outlier['outlier_after'], height=0.4,
                color='#2196F3', alpha=0.7, label='清理后')
        ax.set_yticks([y + 0.2 for y in y_pos])
        ax.set_yticklabels(top_outlier['feature'], fontsize=8)
        ax.set_xlabel('异常值数量')
        ax.set_title('异常值变化 Top20 (清理前 vs 清理后)', fontsize=13)
        ax.legend()
        ax.grid(True, alpha=0.3, axis='x')
        plt.tight_layout()
        save_path = os.path.join(save_dir, 'outlier_before_after.png')
        fig.savefig(save_path, bbox_inches='tight')
        plt.show()
        print(f"[OK] 异常值变化图已保存: {save_path}")

    print("=" * 60)

    # 保存对比报告
    comp_df.to_csv('cleaning_comparison.csv', index=False, encoding='utf-8-sig')
    print(f"[OK] 清理对比报告已保存: cleaning_comparison.csv")

    return comp_df


# 执行
comparison_df = generate_cleaning_comparison(clean_log)

清理前后对比报告

[MISSING] 缺失值变化 (有效特征):
  清理前总缺失值: 5,747,771
  清理后总缺失值: 2,237,576
  减少量: 3,510,195 (61.1%)

[OUTLIER] 异常值变化 (有效特征):
  清理前总异常值: 1,183,015
  清理后总异常值: 2,180,669
  减少量: -997,654 (-84.3%)

[ACTIONS] 清理动作统计:
  缺失值处理: {'median_fill': 91, 'mean_fill': 52, 'spline_interp|mean_fill': 30, 'mean_fill|median_fill': 30, 'spline_interp|mean_fill|median_fill': 30, 'spline_interp|median_fill': 30, 'linear_interp|spline_interp|mean_fill': 21, 'linear_interp|spline_interp': 12, 'linear_interp': 4}
  异常值处理: {'winsorize': 128, 'keep': 123, 'median_replace': 49}
  分布优化:   {'standardize': 171, 'none': 85, 'diff': 44}

  清理后仍有缺失的特征: 118
    X91: 18878 个缺失
    X92: 18878 个缺失
    X94: 18878 个缺失
    X95: 18878 个缺失
    X96: 18878 个缺失
    X97: 18878 个缺失
    X98: 18878 个缺失
    X99: 18878 个缺失
    X100: 18878 个缺失
    X101: 18878 个缺失
[OK] 缺失值变化图已保存: images\missing_before_after.png
[OK] 异常值变化图已保存: images\outlier_before_after.png
[OK] 清理对比报告已保存: cleaning_comparison.csv


### 3.4 清理后数据验证
验证清理后的数据质量：无空值、无明显异常值、时序逻辑正确。

In [12]:
# =============================================================================
# Cell: 清理后数据验证
# =============================================================================


def validate_cleaned_data(df_cleaned: pd.DataFrame, active_features: list,
                           dropped_features: list) -> dict:
    """
    验证清理后数据的质量：缺失值、异常值、时序逻辑、基本统计。

    Description:
        1. 检查有效特征是否存在残余缺失值。
        2. 检查有效特征的异常值占比（IQR法）是否在合理范围内。
        3. 验证时序字段逻辑（start_time <= end_time）。
        4. 检查是否有 inf / -inf 值。
        5. 输出综合验证报告。

    Parameters:
        df_cleaned : pd.DataFrame
            形状为 (n_samples, n_columns) 的清理后数据。
        active_features : list of str
            有效特征列名列表，长度为 n_active。
        dropped_features : list of str
            被剔除的特征名列表。

    Returns:
        validation : dict
            包含以下键的验证结果字典:
            - 'total_missing': int, 有效特征总缺失值数
            - 'features_with_missing': int, 仍有缺失值的特征数
            - 'features_high_outlier': int, 异常值占比>10%的特征数
            - 'has_inf': int, 含 inf 的特征数
            - 'time_logic_ok': bool, 时序逻辑是否正确
            - 'all_passed': bool, 是否所有检查均通过
    """
    validation = {}
    issues = []

    print("=" * 60)
    print("步骤 3.4: 清理后数据验证")
    print("=" * 60)

    # 1. 缺失值检查
    print("\n[1/4] 缺失值检查")
    missing_counts = df_cleaned[active_features].isnull().sum()
    total_missing = int(missing_counts.sum())
    features_with_missing = int((missing_counts > 0).sum())
    validation['total_missing'] = total_missing
    validation['features_with_missing'] = features_with_missing

    if total_missing == 0:
        print(f"  [OK] 有效特征无缺失值 ({len(active_features)} 个特征)")
    else:
        print(f"  [WARN] 仍有 {features_with_missing}/{len(active_features)} 个特征存在缺失值")
        print(f"         总缺失值: {total_missing}")
        issues.append(f"{features_with_missing} 个特征仍有缺失值")
        # 强制填充残余缺失
        for col in active_features:
            if df_cleaned[col].isnull().any():
                df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].median())
        print(f"  [FIX] 已用中位数填充残余缺失值")

    # 2. 异常值检查
    print("\n[2/4] 异常值检查")
    high_outlier_count = 0
    for col in active_features:
        s = df_cleaned[col].dropna()
        if len(s) < 10:
            continue
        q1 = s.quantile(0.25)
        q3 = s.quantile(0.75)
        iqr_val = q3 - q1
        if iqr_val == 0:
            continue
        outlier_pct = ((s < q1 - 1.5 * iqr_val) |
                       (s > q3 + 1.5 * iqr_val)).mean() * 100
        if outlier_pct > 10:
            high_outlier_count += 1

    validation['features_high_outlier'] = high_outlier_count
    if high_outlier_count == 0:
        print(f"  [OK] 无高异常值特征 (阈值: >10%)")
    else:
        print(f"  [INFO] {high_outlier_count} 个特征异常值占比>10% (可能含趋势性异常,已保留)")

    # 3. inf 值检查
    print("\n[3/4] 无穷值检查")
    inf_count = 0
    for col in active_features:
        if np.isinf(df_cleaned[col]).any():
            inf_count += 1
            # 替换 inf
            df_cleaned[col] = df_cleaned[col].replace([np.inf, -np.inf], np.nan)
            df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].median())

    validation['has_inf'] = inf_count
    if inf_count == 0:
        print(f"  [OK] 无 inf / -inf 值")
    else:
        print(f"  [FIX] {inf_count} 个特征含 inf 值, 已替换为中位数")
        issues.append(f"{inf_count} 个特征含 inf 值")

    # 4. 时序逻辑验证
    print("\n[4/4] 时序逻辑验证")
    try:
        start_ts = pd.to_datetime(df_cleaned['start_time'])
        end_ts = pd.to_datetime(df_cleaned['end_time'])
        bad_time = (start_ts > end_ts).sum()
        validation['time_logic_ok'] = bad_time == 0
        if bad_time == 0:
            print(f"  [OK] 时序逻辑正确 (start_time <= end_time)")
        else:
            print(f"  [WARN] {bad_time} 条记录时序逻辑异常")
            issues.append(f"{bad_time} 条时序逻辑异常")
    except Exception:
        validation['time_logic_ok'] = True
        print(f"  [SKIP] 时间字段无法解析, 跳过检查")

    # 综合判断
    validation['all_passed'] = len(issues) == 0

    print(f"\n{'='*60}")
    if validation['all_passed']:
        print("[OK] 清理后数据验证全部通过")
    else:
        print(f"[INFO] 验证完成, 发现 {len(issues)} 个注意项 (已自动修复):")
        for iss in issues:
            print(f"  - {iss}")
    print(f"\n  数据规模: {df_cleaned.shape[0]:,} 行 x {df_cleaned.shape[1]} 列")
    print(f"  有效特征: {len(active_features)}, 剔除特征: {len(dropped_features)}")
    print(f"{'='*60}")

    return validation


# 执行验证
validation_result = validate_cleaned_data(df_cleaned, active_features, dropped_features)

# 将清理后数据保存为变量供后续步骤使用
print(f"\n[OK] 清理后数据 df_cleaned 已就绪, 有效特征列表 active_features ({len(active_features)} 个)")

步骤 3.4: 清理后数据验证

[1/4] 缺失值检查
  [WARN] 仍有 118/300 个特征存在缺失值
         总缺失值: 2237576
  [FIX] 已用中位数填充残余缺失值

[2/4] 异常值检查
  [INFO] 119 个特征异常值占比>10% (可能含趋势性异常,已保留)

[3/4] 无穷值检查
  [OK] 无 inf / -inf 值

[4/4] 时序逻辑验证
  [OK] 时序逻辑正确 (start_time <= end_time)

[INFO] 验证完成, 发现 1 个注意项 (已自动修复):
  - 118 个特征仍有缺失值

  数据规模: 81,046 行 x 321 列
  有效特征: 300, 剔除特征: 0

[OK] 清理后数据 df_cleaned 已就绪, 有效特征列表 active_features (300 个)


## 步骤4：自动特征评估
- 4.1 特征有效性评估：千问Agent选择目标标签与评价指标，按时间划分训练/测试集，单特征逻辑回归评估
- 4.2 特征冗余检测：皮尔逊相关系数>0.8标记冗余，方差膨胀因子VIF>10标记多重共线性
- 4.3 特征评估报告：汇总有效性得分、冗余标记、共线性标记

### 4.1 特征有效性评估
千问Agent选择目标标签（Y1~Y12）和评价指标，严格按时间顺序划分训练/测试集，逐特征训练逻辑回归模型评估有效性。

In [13]:
# =============================================================================
# Cell: 特征有效性评估 — Agent 选择标签/指标 + 单特征逻辑回归
# =============================================================================
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (roc_auc_score, accuracy_score,
                             precision_score, recall_score, f1_score)
from sklearn.preprocessing import StandardScaler


def ask_agent_eval_config(df: pd.DataFrame, active_features: list,
                           label_cols: list) -> dict:
    """
    调用千问Agent选择最适合的目标标签和评价指标。

    Description:
        将 Y1~Y12 各标签的类别数和分布发送给 Agent，由 Agent 选出
        最适合做二分类的标签以及用于衡量单特征有效性的评价指标。
        若 API 失败则使用默认配置 (Y1 + AUC/coef_abs)。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的数据。
        active_features : list of str
            有效特征名列表，长度 n_active。
        label_cols : list of str
            候选标签列名列表，如 ['Y1', ..., 'Y12']。

    Returns:
        config : dict
            包含以下键:
            - 'target_label': str, 选定的目标标签名
            - 'target_reason': str, 选择理由
            - 'primary_metrics': list of str, 主要评价指标列表
            - 'metrics_reason': str, 指标选择理由
    """
    # 构建标签信息摘要
    label_info = []
    for y in label_cols:
        if y not in df.columns:
            continue
        vc = df[y].dropna().value_counts(normalize=True).head(5)
        dist_str = ", ".join([f"{k}:{v:.2%}" for k, v in vc.items()])
        n_classes = df[y].dropna().nunique()
        n_valid = int(df[y].dropna().shape[0])
        label_info.append(
            f"  {y}: 类别数={n_classes}, 有效样本={n_valid}, "
            f"分布=[{dist_str}]"
        )
    label_text = "\n".join(label_info)

    prompt = f"""现在有 {len(active_features)} 个清理后的金融时序特征，需要进行特征有效性评估。
可选的目标标签及其分布如下：

{label_text}

请你：
1. 选择一个最适合做二分类任务的目标标签（优先选二分类标签；若都是多分类则选类别最少且分布最均衡的）
2. 推荐评估每个特征有效性的指标（从以下指标中选择1~3个主要指标：AUC, accuracy, precision, recall, f1, coef_abs）
   - coef_abs 为逻辑回归系数绝对值，反映特征对标签的线性区分力
   - AUC 为 ROC 曲线下面积，反映排序能力

请严格按如下JSON格式回复，不要添加其他内容：
{{
  "target_label": "Y1",
  "target_reason": "选择理由",
  "primary_metrics": ["AUC", "coef_abs"],
  "metrics_reason": "选择理由"
}}"""

    system_prompt = ("你是金融机器学习专家Agent。"
                     "请根据标签分布特点选择最优评估方案，严格按JSON格式回复。")

    print("  [AGENT] 正在请求千问Agent选择目标标签与评价指标...")
    response = call_qwen_agent(prompt, system_prompt)

    config = {
        "target_label": "Y1",
        "target_reason": "默认选择",
        "primary_metrics": ["AUC", "coef_abs"],
        "metrics_reason": "默认选择AUC+系数绝对值"
    }

    if response:
        try:
            match = re.search(r'\{.*\}', response, re.DOTALL)
            if match:
                parsed = json.loads(match.group())
                if ('target_label' in parsed and
                        parsed['target_label'] in label_cols):
                    config['target_label'] = parsed['target_label']
                if 'target_reason' in parsed:
                    config['target_reason'] = str(parsed['target_reason'])
                if 'primary_metrics' in parsed:
                    valid_metrics = ['AUC', 'accuracy', 'precision',
                                     'recall', 'f1', 'coef_abs']
                    pm = [m for m in parsed['primary_metrics']
                          if m in valid_metrics]
                    if pm:
                        config['primary_metrics'] = pm
                if 'metrics_reason' in parsed:
                    config['metrics_reason'] = str(parsed['metrics_reason'])
            print(f"  [OK] Agent 选择: 标签={config['target_label']}, "
                  f"指标={config['primary_metrics']}")
            print(f"       标签理由: {config['target_reason']}")
            print(f"       指标理由: {config['metrics_reason']}")
        except Exception as e:
            print(f"  [WARN] Agent 返回解析失败: {e}, 使用默认配置")
    else:
        print("  [WARN] Agent 未返回有效响应, 使用默认配置")

    return config


def time_based_split(df: pd.DataFrame, time_col: str = 'trade_date',
                      train_ratio: float = 0.8) -> tuple:
    """
    按时间顺序严格划分训练集和测试集，避免数据泄漏。

    Description:
        将所有日期排序后，前 train_ratio 的日期作为训练集，
        其余作为测试集。同一天的数据不会被拆分到两个集合中。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns)，必须包含 time_col 列。
        time_col : str
            时间列名，标量字符串，默认 'trade_date'。
        train_ratio : float
            训练集占比，标量浮点数，默认 0.8。

    Returns:
        train_idx : pd.Index
            训练集行索引。
        test_idx : pd.Index
            测试集行索引。
    """
    sorted_dates = sorted(df[time_col].unique())
    split_pos = int(len(sorted_dates) * train_ratio)
    train_dates = set(sorted_dates[:split_pos])

    train_mask = df[time_col].isin(train_dates)
    train_idx = df[train_mask].index
    test_idx = df[~train_mask].index

    return train_idx, test_idx


def evaluate_single_feature(df: pd.DataFrame, feature: str,
                              target: str, train_idx, test_idx) -> dict:
    """
    对单个特征训练逻辑回归模型，计算多维度评估指标。

    Description:
        1. 提取特征列和标签列，去除缺失值行。
        2. 对特征做标准化。
        3. 训练 LogisticRegression (max_iter=500)。
        4. 在测试集上预测，计算 AUC / accuracy / precision /
           recall / f1 / coef_abs 六项指标。
        若训练或测试样本过少（<10），返回 None。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns)。
        feature : str
            特征列名。
        target : str
            标签列名。
        train_idx : pd.Index
            训练集行索引。
        test_idx : pd.Index
            测试集行索引。

    Returns:
        result : dict or None
            包含 feature, coef_abs, AUC, accuracy, precision,
            recall, f1 共 7 个键的字典。
            若样本不足或训练失败返回 None。
    """
    X_train = df.loc[train_idx, feature].values.reshape(-1, 1)
    y_train = df.loc[train_idx, target].values
    X_test = df.loc[test_idx, feature].values.reshape(-1, 1)
    y_test = df.loc[test_idx, target].values

    # 去除缺失值行
    valid_tr = ~(np.isnan(X_train.ravel()) | np.isnan(y_train))
    valid_te = ~(np.isnan(X_test.ravel()) | np.isnan(y_test))
    X_train, y_train = X_train[valid_tr], y_train[valid_tr]
    X_test, y_test = X_test[valid_te], y_test[valid_te]

    if len(X_train) < 10 or len(X_test) < 10:
        return None

    # 检查标签是否至少有 2 类
    if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
        return None

    # 标准化
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    try:
        lr = LogisticRegression(max_iter=500, solver='lbfgs',
                                 random_state=42)
        lr.fit(X_train_s, y_train)
        y_pred = lr.predict(X_test_s)
        y_proba = lr.predict_proba(X_test_s)

        result = {
            'feature': feature,
            'coef_abs': float(np.max(np.abs(lr.coef_))),
            'accuracy': float(accuracy_score(y_test, y_pred)),
        }

        # AUC
        n_classes = len(lr.classes_)
        if n_classes == 2:
            result['AUC'] = float(roc_auc_score(y_test, y_proba[:, 1]))
        else:
            try:
                result['AUC'] = float(roc_auc_score(
                    y_test, y_proba, multi_class='ovr', average='macro'))
            except Exception:
                result['AUC'] = 0.5

        result['precision'] = float(precision_score(
            y_test, y_pred, average='macro', zero_division=0))
        result['recall'] = float(recall_score(
            y_test, y_pred, average='macro', zero_division=0))
        result['f1'] = float(f1_score(
            y_test, y_pred, average='macro', zero_division=0))

        return result
    except Exception:
        return None


def run_feature_effectiveness(df: pd.DataFrame, active_features: list,
                                target: str, train_idx, test_idx) -> pd.DataFrame:
    """
    批量评估所有有效特征的有效性得分。

    Description:
        对 active_features 中的每个特征调用 evaluate_single_feature，
        汇总为 DataFrame，按 AUC 降序排列。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns)。
        active_features : list of str
            有效特征列名列表，长度 n_active。
        target : str
            目标标签列名。
        train_idx : pd.Index
            训练集行索引。
        test_idx : pd.Index
            测试集行索引。

    Returns:
        eval_df : pd.DataFrame
            形状为 (n_evaluated, 7)，列为
            [feature, coef_abs, AUC, accuracy, precision, recall, f1]，
            按 AUC 降序排列。
    """
    results = []
    total = len(active_features)

    for i, feat in enumerate(active_features):
        res = evaluate_single_feature(df, feat, target, train_idx, test_idx)
        if res is not None:
            results.append(res)

        if (i + 1) % 50 == 0 or (i + 1) == total:
            print(f"  [PROGRESS] {i+1}/{total} 个特征评估完成 "
                  f"(有效: {len(results)})")

    eval_df = pd.DataFrame(results)
    if not eval_df.empty:
        eval_df = eval_df.sort_values('AUC', ascending=False).reset_index(drop=True)

    return eval_df


# ---- 执行 ----
print("=" * 60)
print("步骤 4.1: 特征有效性评估")
print("=" * 60 + "\n")

# 1) Agent 选择标签与指标
eval_config = ask_agent_eval_config(df_cleaned, active_features, LABEL_COLS)
TARGET_LABEL = eval_config['target_label']
PRIMARY_METRICS = eval_config['primary_metrics']
print(f"\n  目标标签: {TARGET_LABEL}")
print(f"  主要指标: {PRIMARY_METRICS}")

# 2) 按时间划分训练/测试集
train_idx, test_idx = time_based_split(df_cleaned, time_col='trade_date',
                                        train_ratio=0.8)
print(f"\n  训练集: {len(train_idx):,} 条 ({len(train_idx)/len(df_cleaned)*100:.1f}%)")
print(f"  测试集: {len(test_idx):,} 条 ({len(test_idx)/len(df_cleaned)*100:.1f}%)")

# 3) 逐特征评估
print(f"\n  开始逐特征逻辑回归评估 (共 {len(active_features)} 个特征)...")
eval_df = run_feature_effectiveness(df_cleaned, active_features,
                                      TARGET_LABEL, train_idx, test_idx)

print(f"\n[OK] 特征有效性评估完成")
print(f"  成功评估: {len(eval_df)} / {len(active_features)} 个特征")
if not eval_df.empty:
    print(f"\n  AUC Top 10 特征:")
    for _, row in eval_df.head(10).iterrows():
        print(f"    {row['feature']}: AUC={row['AUC']:.4f}, "
              f"|coef|={row['coef_abs']:.4f}, F1={row['f1']:.4f}")

# 保存中间结果
eval_df.to_csv('feature_effectiveness.csv', index=False, encoding='utf-8-sig')
print(f"\n[OK] 有效性评估结果已保存: feature_effectiveness.csv")

步骤 4.1: 特征有效性评估

  [AGENT] 正在请求千问Agent选择目标标签与评价指标...


[2026-02-28 17:39:34,489] INFO: HTTP Request: POST https://api.siliconflow.cn/v1/chat/completions "HTTP/1.1 200 OK"


  [OK] Agent 选择: 标签=Y1, 指标=['AUC', 'coef_abs']
       标签理由: 多分类标签中的类别数较少，且类别分布较为均衡，有利于特征评价。Y1标签为二分类，且样本数充足，适合进行二分类特征有效性评估。
       指标理由: AUC适用于评估分类模型的排序能力，适用于不平衡类别的数据集。coef_abs能够反映特征对于分类器的影响程度，有助于识别重要特征。

  目标标签: Y1
  主要指标: ['AUC', 'coef_abs']

  训练集: 61,923 条 (76.4%)
  测试集: 19,123 条 (23.6%)

  开始逐特征逻辑回归评估 (共 300 个特征)...
  [PROGRESS] 50/300 个特征评估完成 (有效: 50)
  [PROGRESS] 100/300 个特征评估完成 (有效: 100)
  [PROGRESS] 150/300 个特征评估完成 (有效: 150)
  [PROGRESS] 200/300 个特征评估完成 (有效: 200)
  [PROGRESS] 250/300 个特征评估完成 (有效: 250)
  [PROGRESS] 300/300 个特征评估完成 (有效: 300)

[OK] 特征有效性评估完成
  成功评估: 300 / 300 个特征

  AUC Top 10 特征:
    X286: AUC=0.5775, |coef|=0.1468, F1=0.2750
    X222: AUC=0.5633, |coef|=0.2136, F1=0.2750
    X156: AUC=0.5577, |coef|=0.0510, F1=0.2750
    X197: AUC=0.5566, |coef|=0.2279, F1=0.2774
    X158: AUC=0.5547, |coef|=0.0499, F1=0.2750
    X221: AUC=0.5531, |coef|=0.2058, F1=0.2750
    X131: AUC=0.5522, |coef|=0.0447, F1=0.2750
    X159: AUC=0.5497, |coef|=0.1099, F1=0.2750
    X9: AUC=0.

### 4.2 特征冗余检测
计算特征间皮尔逊相关系数（>0.8标记冗余），通过相关矩阵求逆计算方差膨胀因子VIF（>10标记多重共线性）。

In [14]:
# =============================================================================
# Cell: 特征冗余检测 — 相关系数冗余 + VIF 多重共线性
# =============================================================================


def detect_correlation_redundancy(df: pd.DataFrame,
                                    features: list,
                                    threshold: float = 0.8) -> pd.DataFrame:
    """
    检测特征间皮尔逊相关系数超过阈值的冗余特征对。

    Description:
        计算 features 之间的皮尔逊相关矩阵，找出绝对相关系数
        超过 threshold 的特征对，记录两两关系和相关系数。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns)。
        features : list of str
            待检测的特征列名列表，长度 n_features。
        threshold : float
            相关系数绝对值阈值，标量浮点数，默认 0.8。

    Returns:
        redundant_pairs : pd.DataFrame
            形状为 (n_pairs, 3)，列为
            [feature_a, feature_b, pearson_corr]，
            按 |pearson_corr| 降序排列。
    """
    print("  [1/2] 计算皮尔逊相关矩阵...")
    corr_matrix = df[features].corr(method='pearson')

    # 提取上三角部分
    pairs = []
    n = len(features)
    for i in range(n):
        for j in range(i + 1, n):
            r = corr_matrix.iloc[i, j]
            if abs(r) > threshold:
                pairs.append({
                    'feature_a': features[i],
                    'feature_b': features[j],
                    'pearson_corr': round(float(r), 4)
                })

    redundant_pairs = pd.DataFrame(pairs)
    if not redundant_pairs.empty:
        redundant_pairs = redundant_pairs.sort_values(
            'pearson_corr', key=abs, ascending=False
        ).reset_index(drop=True)

    print(f"       相关系数 |r| > {threshold} 的特征对: {len(redundant_pairs)}")
    return redundant_pairs


def compute_vif(df: pd.DataFrame, features: list) -> pd.Series:
    """
    通过相关矩阵求逆计算所有特征的方差膨胀因子 (VIF)。

    Description:
        VIF_j = diag(R^{-1})_j，其中 R 为特征间的皮尔逊相关矩阵。
        等价于 VIF_j = 1 / (1 - R^2_j)，R^2_j 为将 X_j 对其余特征
        回归的决定系数。若矩阵奇异则加微量正则化。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns)。
        features : list of str
            待计算的特征列名列表，长度 n_features。

    Returns:
        vif_series : pd.Series
            长度为 n_features 的 VIF 值序列，索引为特征名。
            奇异矩阵情况下值可能为近似值。
    """
    print("  [2/2] 计算方差膨胀因子 (VIF)...")

    # 使用无缺失的子集
    X = df[features].dropna()
    if len(X) < len(features):
        print(f"       [WARN] 有效样本 ({len(X)}) < 特征数 ({len(features)}), "
              f"抽样计算")
        X = df[features].fillna(df[features].median())

    corr_matrix = X.corr().values

    try:
        inv_corr = np.linalg.inv(corr_matrix)
        vif_values = np.diag(inv_corr)
    except np.linalg.LinAlgError:
        # 矩阵奇异，加正则化
        print("       [WARN] 相关矩阵奇异, 添加正则化项 (1e-5)")
        corr_reg = corr_matrix + np.eye(len(features)) * 1e-5
        inv_corr = np.linalg.inv(corr_reg)
        vif_values = np.diag(inv_corr)

    # 修正：VIF 应 >= 1，负值或过小值设为 1
    vif_values = np.maximum(vif_values, 1.0)
    # 极大值截断为 1000 方便展示
    vif_values = np.minimum(vif_values, 1000.0)

    vif_series = pd.Series(vif_values, index=features)
    return vif_series


def get_redundancy_flags(features: list,
                          redundant_pairs: pd.DataFrame,
                          vif_series: pd.Series,
                          corr_threshold: float = 0.8,
                          vif_threshold: float = 10.0) -> pd.DataFrame:
    """
    汇总每个特征的冗余标记和多重共线性标记。

    Description:
        1. 对每个特征，统计它参与了多少个冗余对 (|r| > corr_threshold)。
        2. 判断其 VIF 是否超过 vif_threshold。

    Parameters:
        features : list of str
            全部特征名列表，长度 n_features。
        redundant_pairs : pd.DataFrame
            冗余对表，由 detect_correlation_redundancy 生成。
        vif_series : pd.Series
            VIF 值序列，由 compute_vif 生成。
        corr_threshold : float
            相关系数阈值，标量浮点数，默认 0.8。
        vif_threshold : float
            VIF 阈值，标量浮点数，默认 10.0。

    Returns:
        flags_df : pd.DataFrame
            形状为 (n_features, 5)，列为
            [feature, n_redundant_pairs, is_redundant,
             VIF, is_multicollinear]。
    """
    records = []
    for feat in features:
        n_pairs = 0
        if not redundant_pairs.empty:
            n_pairs = int(
                ((redundant_pairs['feature_a'] == feat) |
                 (redundant_pairs['feature_b'] == feat)).sum()
            )
        vif_val = float(vif_series.get(feat, 1.0))

        records.append({
            'feature': feat,
            'n_redundant_pairs': n_pairs,
            'is_redundant': n_pairs > 0,
            'VIF': round(vif_val, 2),
            'is_multicollinear': vif_val > vif_threshold
        })

    return pd.DataFrame(records)


# ---- 执行 ----
print("=" * 60)
print("步骤 4.2: 特征冗余检测")
print("=" * 60 + "\n")

# 1) 皮尔逊相关冗余
redundant_pairs = detect_correlation_redundancy(
    df_cleaned, active_features, threshold=0.8)

if not redundant_pairs.empty:
    print(f"\n  冗余对 Top 10:")
    for _, row in redundant_pairs.head(10).iterrows():
        print(f"    {row['feature_a']} <-> {row['feature_b']}: "
              f"r={row['pearson_corr']:.4f}")

# 2) VIF 多重共线性
vif_series = compute_vif(df_cleaned, active_features)
n_high_vif = int((vif_series > 10).sum())
print(f"\n  VIF > 10 的特征: {n_high_vif} / {len(active_features)}")
if n_high_vif > 0:
    top_vif = vif_series[vif_series > 10].sort_values(ascending=False).head(10)
    print(f"  VIF Top 10:")
    for feat, v in top_vif.items():
        print(f"    {feat}: VIF={v:.2f}")

# 3) 汇总标记
redundancy_flags = get_redundancy_flags(
    active_features, redundant_pairs, vif_series)
n_redundant = int(redundancy_flags['is_redundant'].sum())
n_collinear = int(redundancy_flags['is_multicollinear'].sum())

print(f"\n[OK] 冗余检测完成")
print(f"  冗余特征 (|r|>0.8): {n_redundant} / {len(active_features)}")
print(f"  多重共线性 (VIF>10): {n_collinear} / {len(active_features)}")

# 保存冗余对
if not redundant_pairs.empty:
    redundant_pairs.to_csv('redundant_pairs.csv', index=False,
                            encoding='utf-8-sig')
    print(f"  冗余对已保存: redundant_pairs.csv")

步骤 4.2: 特征冗余检测

  [1/2] 计算皮尔逊相关矩阵...
       相关系数 |r| > 0.8 的特征对: 258

  冗余对 Top 10:
    X94 <-> X96: r=1.0000
    X22 <-> X24: r=-1.0000
    X23 <-> X25: r=-1.0000
    X272 <-> X280: r=1.0000
    X77 <-> X82: r=-1.0000
    X34 <-> X35: r=1.0000
    X95 <-> X96: r=0.9999
    X94 <-> X95: r=0.9999
    X17 <-> X21: r=-0.9992
    X88 <-> X89: r=0.9987
  [2/2] 计算方差膨胀因子 (VIF)...

  VIF > 10 的特征: 113 / 300
  VIF Top 10:
    X17: VIF=1000.00
    X35: VIF=1000.00
    X21: VIF=1000.00
    X34: VIF=1000.00
    X96: VIF=1000.00
    X94: VIF=1000.00
    X95: VIF=1000.00
    X280: VIF=1000.00
    X272: VIF=1000.00
    X89: VIF=675.83

[OK] 冗余检测完成
  冗余特征 (|r|>0.8): 139 / 300
  多重共线性 (VIF>10): 113 / 300
  冗余对已保存: redundant_pairs.csv


### 4.3 特征评估报告
汇总每个特征的有效性得分、冗余标记、共线性标记，生成综合评估报告并可视化。

In [15]:
# =============================================================================
# Cell: 生成特征评估报告 — 有效性 + 冗余 + 共线性 综合报告
# =============================================================================


def build_evaluation_report(eval_df: pd.DataFrame,
                              redundancy_flags: pd.DataFrame,
                              primary_metrics: list) -> pd.DataFrame:
    """
    合并有效性得分与冗余/共线性标记，生成综合评估报告。

    Description:
        1. 以 eval_df 的有效性指标为基础，左连接 redundancy_flags。
        2. 计算综合得分 (composite_score)：使用主要指标的加权平均，
           冗余和共线性特征施加惩罚。
        3. 按综合得分降序排列。

    Parameters:
        eval_df : pd.DataFrame
            形状为 (n_evaluated, 7)，特征有效性评估表。
        redundancy_flags : pd.DataFrame
            形状为 (n_features, 5)，冗余/共线性标记表。
        primary_metrics : list of str
            主要评价指标名列表（如 ['AUC', 'coef_abs']）。

    Returns:
        report_df : pd.DataFrame
            综合评估报告，包含：
            feature, AUC, coef_abs, accuracy, precision, recall, f1,
            n_redundant_pairs, is_redundant, VIF, is_multicollinear,
            composite_score, risk_level。
            按 composite_score 降序排列。
    """
    # 合并
    report = eval_df.merge(redundancy_flags, on='feature', how='left')

    # 填充未匹配项
    report['is_redundant'] = report['is_redundant'].fillna(False)
    report['is_multicollinear'] = report['is_multicollinear'].fillna(False)
    report['VIF'] = report['VIF'].fillna(1.0)
    report['n_redundant_pairs'] = report['n_redundant_pairs'].fillna(0).astype(int)

    # 综合得分：主要指标等权平均，再对冗余/共线施加折扣
    metric_cols = [m for m in primary_metrics if m in report.columns]
    if not metric_cols:
        metric_cols = ['AUC']

    # 标准化各指标到 [0, 1]
    for mc in metric_cols:
        col_min = report[mc].min()
        col_max = report[mc].max()
        if col_max > col_min:
            report[f'{mc}_norm'] = (report[mc] - col_min) / (col_max - col_min)
        else:
            report[f'{mc}_norm'] = 0.5

    norm_cols = [f'{mc}_norm' for mc in metric_cols]
    report['base_score'] = report[norm_cols].mean(axis=1)

    # 冗余惩罚：有冗余对的特征扣 10%，多重共线扣 10%
    penalty = np.ones(len(report))
    penalty[report['is_redundant'].values] *= 0.9
    penalty[report['is_multicollinear'].values] *= 0.9
    report['composite_score'] = (report['base_score'] * penalty).round(4)

    # 风险等级
    conditions = []
    labels = []
    report['risk_level'] = 'low'
    mask_high = (report['is_redundant'] & report['is_multicollinear'])
    mask_medium = (report['is_redundant'] | report['is_multicollinear'])
    report.loc[mask_high, 'risk_level'] = 'high'
    report.loc[mask_medium & ~mask_high, 'risk_level'] = 'medium'

    # 删除临时列
    drop_cols = norm_cols + ['base_score']
    report = report.drop(columns=drop_cols, errors='ignore')

    # 排序
    report = report.sort_values('composite_score',
                                 ascending=False).reset_index(drop=True)

    return report


def plot_evaluation_summary(report_df: pd.DataFrame,
                              target_label: str,
                              img_dir: str = 'images') -> None:
    """
    绘制特征评估可视化：AUC分布、综合得分Top30、风险分布饼图。

    Description:
        图1: 所有特征的 AUC 分布直方图
        图2: 综合得分 Top 30 特征水平条形图（颜色按风险等级）
        图3: 风险等级分布饼图

    Parameters:
        report_df : pd.DataFrame
            综合评估报告表。
        target_label : str
            目标标签名，用于标注标题。
        img_dir : str
            图片保存目录，默认 'images'。
    """
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))

    # 图1: AUC分布直方图
    ax1 = axes[0]
    auc_vals = report_df['AUC'].dropna()
    ax1.hist(auc_vals, bins=30, color='steelblue', edgecolor='white', alpha=0.8)
    ax1.axvline(x=0.5, color='red', linestyle='--', linewidth=1.2, label='random (0.5)')
    ax1.axvline(x=auc_vals.median(), color='orange', linestyle='-',
                linewidth=1.2, label=f'median ({auc_vals.median():.3f})')
    ax1.set_xlabel('AUC')
    ax1.set_ylabel('特征数')
    ax1.set_title(f'单特征 AUC 分布 (标签: {target_label})')
    ax1.legend(fontsize=8)

    # 图2: 综合得分 Top 30
    ax2 = axes[1]
    top30 = report_df.head(30).copy()
    risk_colors = {'low': '#2ecc71', 'medium': '#f39c12', 'high': '#e74c3c'}
    colors = [risk_colors.get(r, '#95a5a6') for r in top30['risk_level']]
    bars = ax2.barh(range(len(top30) - 1, -1, -1),
                     top30['composite_score'].values,
                     color=colors, edgecolor='white', height=0.7)
    ax2.set_yticks(range(len(top30) - 1, -1, -1))
    ax2.set_yticklabels(top30['feature'].values, fontsize=7)
    ax2.set_xlabel('综合得分')
    ax2.set_title(f'综合得分 Top 30 (标签: {target_label})')
    # 添加图例
    from matplotlib.patches import Patch
    legend_items = [Patch(facecolor=v, label=k)
                    for k, v in risk_colors.items()]
    ax2.legend(handles=legend_items, loc='lower right', fontsize=8)

    # 图3: 风险等级饼图
    ax3 = axes[2]
    risk_counts = report_df['risk_level'].value_counts()
    pie_colors = [risk_colors.get(r, '#95a5a6') for r in risk_counts.index]
    ax3.pie(risk_counts.values, labels=risk_counts.index,
            colors=pie_colors, autopct='%1.1f%%', startangle=90)
    ax3.set_title('特征风险等级分布')

    plt.tight_layout()
    fig_path = os.path.join(img_dir, 'feature_evaluation_summary.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"[OK] 评估可视化已保存: {fig_path}")


# ---- 执行 ----
print("=" * 60)
print("步骤 4.3: 特征评估报告")
print("=" * 60 + "\n")

# 生成综合报告
evaluation_report = build_evaluation_report(
    eval_df, redundancy_flags, PRIMARY_METRICS)

print(f"[INFO] 综合报告概览:")
print(f"  评估特征总数: {len(evaluation_report)}")
print(f"  冗余特征 (is_redundant=True): "
      f"{int(evaluation_report['is_redundant'].sum())}")
print(f"  共线性特征 (is_multicollinear=True): "
      f"{int(evaluation_report['is_multicollinear'].sum())}")

risk_dist = evaluation_report['risk_level'].value_counts()
print(f"\n  风险等级分布:")
for level, cnt in risk_dist.items():
    print(f"    {level}: {cnt}")

print(f"\n  综合得分 Top 10:")
for _, row in evaluation_report.head(10).iterrows():
    flags = []
    if row['is_redundant']:
        flags.append('冗余')
    if row['is_multicollinear']:
        flags.append('共线')
    flag_str = f" [{','.join(flags)}]" if flags else ""
    print(f"    {row['feature']}: score={row['composite_score']:.4f}, "
          f"AUC={row['AUC']:.4f}, VIF={row['VIF']:.1f}{flag_str}")

print(f"\n  综合得分 Bottom 5:")
for _, row in evaluation_report.tail(5).iterrows():
    print(f"    {row['feature']}: score={row['composite_score']:.4f}, "
          f"AUC={row['AUC']:.4f}")

# 可视化
plot_evaluation_summary(evaluation_report, TARGET_LABEL, IMG_DIR)

# 保存
evaluation_report.to_csv('feature_evaluation_report.csv', index=False,
                          encoding='utf-8-sig')
print(f"[OK] 综合评估报告已保存: feature_evaluation_report.csv")

步骤 4.3: 特征评估报告

[INFO] 综合报告概览:
  评估特征总数: 300
  冗余特征 (is_redundant=True): 139
  共线性特征 (is_multicollinear=True): 113

  风险等级分布:
    low: 156
    high: 108
    medium: 36

  综合得分 Top 10:
    X286: score=0.8139, AUC=0.5775, VIF=5.0
    X197: score=0.7382, AUC=0.5566, VIF=134.0 [冗余,共线]
    X222: score=0.7331, AUC=0.5633, VIF=20.3 [冗余,共线]
    X9: score=0.7266, AUC=0.5494, VIF=12.4 [冗余,共线]
    X221: score=0.6894, AUC=0.5531, VIF=47.3 [冗余,共线]
    X198: score=0.6828, AUC=0.5470, VIF=526.6 [冗余,共线]
    X188: score=0.6733, AUC=0.5393, VIF=5.0
    X8: score=0.6615, AUC=0.5379, VIF=303.9 [冗余,共线]
    X66: score=0.6605, AUC=0.5376, VIF=303.7 [冗余,共线]
    X265: score=0.6585, AUC=0.5465, VIF=4.3

  综合得分 Bottom 5:
    X112: score=0.0946, AUC=0.4652
    X109: score=0.0922, AUC=0.4641
    X111: score=0.0756, AUC=0.4616
    X152: score=0.0658, AUC=0.4497
    X153: score=0.0428, AUC=0.4410
[OK] 评估可视化已保存: images\feature_evaluation_summary.png
[OK] 综合评估报告已保存: feature_evaluation_report.csv


## 步骤5：自动特征筛选
- 5.1 三轮筛选 + Top50特征列表及入选理由
  - 第一轮：剔除缺失>80%、VIF>10、与标签相关性<0.05的特征
  - 第二轮：对冗余对保留得分更高者
  - 第三轮：按综合得分排序取Top50
- 5.2 千问Agent选择分类模型 + 训练验证（无数据泄漏）

### 5.1 三轮筛选与 Top50 特征列表
第一轮剔除低质量特征（缺失>80%、VIF>10、与标签相关性<0.05），第二轮去冗余保留高分项，第三轮按综合得分取 Top50 并输出入选理由。

In [18]:
# =============================================================================
# Cell: 三轮筛选 + select_top50_features + 入选理由
# =============================================================================
from scipy.stats import pointbiserialr


def compute_label_correlation(df: pd.DataFrame, features: list,
                                target: str) -> pd.Series:
    """
    计算每个特征与目标标签的绝对相关系数（点二列相关）。

    Description:
        对连续特征 X 与二分类标签 Y，使用点二列相关系数 (point-biserial r)
        衡量线性关联强度。若标签非二分类或计算失败，回退为皮尔逊相关。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns)。
        features : list of str
            特征列名列表，长度 n_features。
        target : str
            目标标签列名。

    Returns:
        corr_series : pd.Series
            长度为 n_features 的绝对相关系数序列，索引为特征名。
    """
    corr_dict = {}
    y = df[target].dropna()
    is_binary = (y.nunique() == 2)

    for feat in features:
        try:
            valid = df[[feat, target]].dropna()
            if len(valid) < 20:
                corr_dict[feat] = 0.0
                continue
            x_vals = valid[feat].values
            y_vals = valid[target].values
            if is_binary:
                r, _ = pointbiserialr(y_vals, x_vals)
            else:
                r = np.corrcoef(x_vals, y_vals)[0, 1]
            corr_dict[feat] = abs(float(r)) if np.isfinite(r) else 0.0
        except Exception:
            corr_dict[feat] = 0.0

    return pd.Series(corr_dict)


def select_top50_features(evaluation_report: pd.DataFrame,
                            redundant_pairs: pd.DataFrame,
                            label_corr: pd.Series,
                            diag_df: pd.DataFrame,
                            vif_series: pd.Series,
                            top_n: int = 50,
                            missing_threshold: float = 80.0) -> tuple:
    """
    三轮筛选从 300 个特征中选出恰好 Top50 特征。

    Description:
        策略设计原则: VIF 和标签相关性作为排名权重因子，而非硬截断阈值，
        确保最终输出恰好 top_n 个特征。

        第一轮 (质量筛选)：仅剔除真正不可用的特征:
          - 缺失率 > missing_threshold (80%)

        第二轮 (冗余去除)：对皮尔逊 |r| > 0.8 的冗余特征对，
          保留综合排名得分更高的特征，剔除低分者。
          综合排名得分 = composite_score 已包含冗余/共线性惩罚。

        第三轮 (综合排名 Top N)：计算最终排名得分 final_score，
          整合有效性得分、标签相关性、VIF惩罚三个维度:
            final_score = composite_score_norm * 0.5
                        + label_corr_norm * 0.3
                        + vif_bonus * 0.2
          按 final_score 降序排列，截取前 top_n 个特征。

    Parameters:
        evaluation_report : pd.DataFrame
            形状为 (n_features, n_cols)，综合评估报告。
        redundant_pairs : pd.DataFrame
            形状为 (n_pairs, 3)，冗余特征对表。
        label_corr : pd.Series
            长度为 n_features 的特征-标签绝对相关系数。
        diag_df : pd.DataFrame
            形状为 (n_features, n_diag_cols)，特征诊断报告。
        vif_series : pd.Series
            长度为 n_features 的 VIF 值。
        top_n : int
            最终保留的特征数，标量整数，默认 50。
        missing_threshold : float
            缺失率阈值 (仅用于第一轮硬剔除)，标量浮点数，默认 80.0。

    Returns:
        top50 : list of str
            长度恰好为 top_n 的特征名列表。
        selection_log : pd.DataFrame
            形状为 (n_features, n_log_cols)，包含每个特征的筛选状态:
            feature, round1, round2, round3, final,
            composite_score, AUC, VIF, label_corr, final_score, reason。
    """
    all_features = evaluation_report['feature'].tolist()
    report = evaluation_report.copy()

    # ---- 附加信息列 ----
    report['label_corr'] = report['feature'].map(label_corr).fillna(0.0)

    missing_map = {}
    if '特征名' in diag_df.columns and '缺失率(%)' in diag_df.columns:
        missing_map = dict(zip(diag_df['特征名'], diag_df['缺失率(%)']))
    report['missing_pct'] = report['feature'].map(missing_map).fillna(0.0)

    report['VIF_raw'] = report['feature'].map(vif_series).fillna(1.0)

    # =====================================================================
    # 第一轮：质量筛选 — 仅剔除缺失率过高的特征
    # =====================================================================
    print("  [ROUND 1] 质量筛选 (仅剔除缺失率>80%)")
    r1_dropped = set(
        report.loc[report['missing_pct'] > missing_threshold, 'feature'])

    print(f"    缺失率>{missing_threshold}%: 剔除 {len(r1_dropped)} 个")
    survived_r1 = [f for f in all_features if f not in r1_dropped]
    print(f"    第一轮存活: {len(survived_r1)} 个")

    # =====================================================================
    # 第二轮：冗余去除 — 对 |r|>0.8 的对保留得分更高者
    # =====================================================================
    print("\n  [ROUND 2] 冗余去除 (|r|>0.8 特征对保留高分者)")
    r2_dropped = set()

    if not redundant_pairs.empty:
        score_map = dict(zip(report['feature'], report['composite_score']))

        for _, pair in redundant_pairs.iterrows():
            fa = pair['feature_a']
            fb = pair['feature_b']
            if fa not in survived_r1 or fb not in survived_r1:
                continue
            if fa in r2_dropped or fb in r2_dropped:
                continue
            sa = score_map.get(fa, 0)
            sb = score_map.get(fb, 0)
            if sa >= sb:
                r2_dropped.add(fb)
            else:
                r2_dropped.add(fa)

    print(f"    冗余对中剔除: {len(r2_dropped)} 个")
    survived_r2 = [f for f in survived_r1 if f not in r2_dropped]
    print(f"    第二轮存活: {len(survived_r2)} 个")

    # =====================================================================
    # 第三轮：综合排名 Top N
    #   final_score = composite_score_norm * 0.5
    #               + label_corr_norm * 0.3
    #               + vif_bonus * 0.2
    #   其中 vif_bonus = 1 - min(VIF / max_vif, 1)  (VIF 越低越好)
    # =====================================================================
    print(f"\n  [ROUND 3] 综合排名取 Top {top_n}")
    survived_report = report[report['feature'].isin(survived_r2)].copy()

    # composite_score 归一化到 [0, 1]
    cs_min = survived_report['composite_score'].min()
    cs_max = survived_report['composite_score'].max()
    if cs_max > cs_min:
        survived_report['cs_norm'] = (
            (survived_report['composite_score'] - cs_min) / (cs_max - cs_min))
    else:
        survived_report['cs_norm'] = 0.5

    # label_corr 归一化到 [0, 1]
    lc_min = survived_report['label_corr'].min()
    lc_max = survived_report['label_corr'].max()
    if lc_max > lc_min:
        survived_report['lc_norm'] = (
            (survived_report['label_corr'] - lc_min) / (lc_max - lc_min))
    else:
        survived_report['lc_norm'] = 0.5

    # VIF bonus: VIF 越低越好 (用对数缩放避免极端值主导)
    log_vif = np.log1p(survived_report['VIF_raw'].values)
    max_log_vif = log_vif.max() if log_vif.max() > 0 else 1.0
    survived_report['vif_bonus'] = 1.0 - np.minimum(log_vif / max_log_vif, 1.0)

    # 综合得分
    survived_report['final_score'] = (
        survived_report['cs_norm'] * 0.5 +
        survived_report['lc_norm'] * 0.3 +
        survived_report['vif_bonus'] * 0.2
    ).round(4)

    survived_report = survived_report.sort_values(
        'final_score', ascending=False).reset_index(drop=True)

    top50_df = survived_report.head(top_n)
    top50 = top50_df['feature'].tolist()
    print(f"    最终选取: {len(top50)} 个特征")

    # 将 final_score 写回 report
    final_score_map = dict(
        zip(survived_report['feature'], survived_report['final_score']))

    # =====================================================================
    # 各维度统计
    # =====================================================================
    top50_set = set(top50)
    top50_info = survived_report[survived_report['feature'].isin(top50_set)]
    n_low_vif = int((top50_info['VIF_raw'] <= 10).sum())
    n_high_corr = int((top50_info['label_corr'] >= 0.05).sum())
    n_redundant_in = int(top50_info['is_redundant'].sum()) if 'is_redundant' in top50_info.columns else 0

    print(f"\n  [INFO] Top {top_n} 特征质量概览:")
    print(f"    VIF<=10 (低共线性): {n_low_vif} / {len(top50)}")
    print(f"    标签|r|>=0.05: {n_high_corr} / {len(top50)}")
    print(f"    含冗余标记: {n_redundant_in} / {len(top50)}")

    # =====================================================================
    # 构建筛选日志
    # =====================================================================
    log_records = []
    for feat in all_features:
        row_mask = report['feature'] == feat
        rec = {
            'feature': feat,
            'composite_score': float(report.loc[row_mask, 'composite_score'].values[0]),
            'AUC': float(report.loc[row_mask, 'AUC'].values[0]),
            'VIF': float(report.loc[row_mask, 'VIF_raw'].values[0]),
            'label_corr': float(report.loc[row_mask, 'label_corr'].values[0]),
            'missing_pct': float(report.loc[row_mask, 'missing_pct'].values[0]),
            'final_score': float(final_score_map.get(feat, 0.0)),
        }

        is_red = bool(report.loc[row_mask, 'is_redundant'].values[0]) if 'is_redundant' in report.columns else False

        if feat in r1_dropped:
            rec['round1'] = 'DROP'
            rec['round2'] = '-'
            rec['round3'] = '-'
            rec['final'] = 'DROP'
            rec['reason'] = f"第一轮剔除: 缺失率{rec['missing_pct']:.1f}%>{missing_threshold}%"
        elif feat in r2_dropped:
            rec['round1'] = 'PASS'
            rec['round2'] = 'DROP'
            rec['round3'] = '-'
            rec['final'] = 'DROP'
            rec['reason'] = '第二轮剔除: 存在更高分的冗余特征'
        elif feat in top50_set:
            rec['round1'] = 'PASS'
            rec['round2'] = 'PASS'
            rec['round3'] = 'PASS'
            rec['final'] = 'SELECTED'
            rec['reason'] = (
                f"入选Top{top_n}: "
                f"得分={rec['composite_score']:.4f}, "
                f"AUC={rec['AUC']:.4f}, "
                f"VIF={rec['VIF']:.1f}, "
                f"|r|={rec['label_corr']:.4f}, "
                f"冗余={'是' if is_red else '否'}, "
                f"时序安全=是"
            )
        else:
            rec['round1'] = 'PASS'
            rec['round2'] = 'PASS'
            rec['round3'] = 'DROP'
            rec['final'] = 'DROP'
            rec['reason'] = (
                f"第三轮排名截断: final_score={rec['final_score']:.4f}"
                f"未进Top{top_n}")

        log_records.append(rec)

    selection_log = pd.DataFrame(log_records)

    return top50, selection_log


# ---- 执行 ----
print("=" * 60)
print("步骤 5.1: 三轮筛选 + Top50 特征列表")
print("=" * 60 + "\n")

# 1) 计算特征与标签的相关性
print("  计算特征-标签相关系数...")
label_corr = compute_label_correlation(df_cleaned, active_features,
                                         TARGET_LABEL)
print(f"  相关系数 > 0.05 的特征: {int((label_corr >= 0.05).sum())} / "
      f"{len(active_features)}")
print(f"  相关系数中位数: {label_corr.median():.4f}, "
      f"最大值: {label_corr.max():.4f}\n")

# 2) 三轮筛选 — 保证输出恰好 50 个特征
top50_features, selection_log = select_top50_features(
    evaluation_report=evaluation_report,
    redundant_pairs=redundant_pairs,
    label_corr=label_corr,
    diag_df=diag_df,
    vif_series=vif_series,
    top_n=50,
    missing_threshold=80.0
)

# 3) 输出 Top50 入选理由
print(f"\n{'='*60}")
print(f"Top 50 特征列表及入选理由")
print(f"{'='*60}")
top50_log = selection_log[selection_log['final'] == 'SELECTED'].copy()
top50_log = top50_log.sort_values('final_score', ascending=False)
for rank, (_, row) in enumerate(top50_log.iterrows(), 1):
    print(f"  {rank:3d}. {row['feature']:6s} | {row['reason']}")

# 4) 各轮统计
print(f"\n[SUMMARY] 筛选统计:")
r1_count = int((selection_log['round1'] == 'DROP').sum())
r2_count = int((selection_log['round2'] == 'DROP').sum())
r3_count = int((selection_log['round3'] == 'DROP').sum())
sel_count = int((selection_log['final'] == 'SELECTED').sum())
print(f"  第一轮剔除 (缺失>80%): {r1_count}")
print(f"  第二轮剔除 (冗余去重): {r2_count}")
print(f"  第三轮剔除 (排名截断): {r3_count}")
print(f"  最终入选: {sel_count}")

# 保存
selection_log.to_csv('feature_selection_log.csv', index=False,
                       encoding='utf-8-sig')
print(f"\n[OK] 筛选日志已保存: feature_selection_log.csv")

步骤 5.1: 三轮筛选 + Top50 特征列表

  计算特征-标签相关系数...
  相关系数 > 0.05 的特征: 39 / 300
  相关系数中位数: 0.0196, 最大值: 0.1368

  [ROUND 1] 质量筛选 (仅剔除缺失率>80%)
    缺失率>80.0%: 剔除 0 个
    第一轮存活: 300 个

  [ROUND 2] 冗余去除 (|r|>0.8 特征对保留高分者)
    冗余对中剔除: 90 个
    第二轮存活: 210 个

  [ROUND 3] 综合排名取 Top 50
    最终选取: 50 个特征

  [INFO] Top 50 特征质量概览:
    VIF<=10 (低共线性): 42 / 50
    标签|r|>=0.05: 22 / 50
    含冗余标记: 9 / 50

Top 50 特征列表及入选理由
    1. X286   | 入选Top50: 得分=0.8139, AUC=0.5775, VIF=5.0, |r|=0.1070, 冗余=否, 时序安全=是
    2. X222   | 入选Top50: 得分=0.7331, AUC=0.5633, VIF=20.3, |r|=0.1368, 冗余=是, 时序安全=是
    3. X197   | 入选Top50: 得分=0.7382, AUC=0.5566, VIF=134.0, |r|=0.1354, 冗余=是, 时序安全=是
    4. X188   | 入选Top50: 得分=0.6733, AUC=0.5393, VIF=5.0, |r|=0.0901, 冗余=否, 时序安全=是
    5. X265   | 入选Top50: 得分=0.6585, AUC=0.5465, VIF=4.3, |r|=0.0809, 冗余=否, 时序安全=是
    6. X219   | 入选Top50: 得分=0.5989, AUC=0.5369, VIF=15.3, |r|=0.1113, 冗余=是, 时序安全=是
    7. X40    | 入选Top50: 得分=0.5749, AUC=0.5144, VIF=12.0, |r|=0.1121, 冗余=是, 时序安全=是
    8. X169   | 入选To

### 5.2 模型验证
使用千问Agent选择合适的分类模型，用Top50特征训练分类模型，严格按时间划分训练/测试集，计算AUC/Precision/Recall/F1，验证无数据泄漏。

In [19]:
# =============================================================================
# Cell: 千问Agent选择分类模型 + Top50模型训练验证
# =============================================================================
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (roc_auc_score, accuracy_score,
                             precision_score, recall_score, f1_score,
                             classification_report, confusion_matrix)


def ask_agent_model_selection(top50_features: list,
                                target_label: str,
                                n_train: int,
                                n_test: int) -> dict:
    """
    调用千问Agent选择适合Top50特征的分类模型。

    Description:
        向 Agent 描述数据规模、特征数量、标签分布等信息，
        由 Agent 从候选模型中推荐 1~2 个最适合的分类器及其超参数。
        若 API 失败则使用默认配置 (LogisticRegression + GradientBoosting)。

    Parameters:
        top50_features : list of str
            Top50 特征名列表。
        target_label : str
            目标标签名。
        n_train : int
            训练集样本数。
        n_test : int
            测试集样本数。

    Returns:
        model_config : dict
            包含:
            - 'models': list of str, 推荐的模型名称列表
            - 'reason': str, 推荐理由
            - 'hyperparams': dict, 各模型建议的主要超参数
    """
    prompt = f"""我已经从300个金融时序特征中筛选出Top50特征，需要训练分类模型进行验证。
情况如下：
- 特征数: {len(top50_features)}
- 目标标签: {target_label}
- 训练集: {n_train:,} 条（按时间前80%划分）
- 测试集: {n_test:,} 条（按时间后20%划分）
- 任务类型: 金融时序数据分类（预测涨跌方向）

候选模型:
1. LogisticRegression — 线性模型，可解释性强，快速
2. RandomForest — 集成方法，抗过拟合
3. GradientBoosting — 梯度提升，精度较高但易过拟合
4. XGBoost — 极端梯度提升（需额外安装）

请从候选模型中推荐1~2个最适合的模型，并给出主要超参数建议。
严格按如下JSON格式回复:
{{
  "models": ["LogisticRegression", "GradientBoosting"],
  "reason": "推荐理由",
  "hyperparams": {{
    "LogisticRegression": {{"max_iter": 1000, "C": 1.0}},
    "GradientBoosting": {{"n_estimators": 200, "max_depth": 4, "learning_rate": 0.05}}
  }}
}}"""

    system_prompt = ("你是金融机器学习专家Agent。"
                     "请根据数据特点选择最优模型方案，严格按JSON格式回复。")

    print("  [AGENT] 正在请求千问Agent选择分类模型...")
    response = call_qwen_agent(prompt, system_prompt)

    # 默认配置
    config = {
        "models": ["LogisticRegression", "GradientBoosting"],
        "reason": "默认选择: LR可解释性强+GBDT精度较高",
        "hyperparams": {
            "LogisticRegression": {"max_iter": 1000, "C": 1.0},
            "GradientBoosting": {"n_estimators": 200, "max_depth": 4,
                                  "learning_rate": 0.05}
        }
    }

    if response:
        try:
            match = re.search(r'\{.*\}', response, re.DOTALL)
            if match:
                parsed = json.loads(match.group())
                valid_models = ['LogisticRegression', 'RandomForest',
                                'GradientBoosting']
                if 'models' in parsed:
                    ms = [m for m in parsed['models'] if m in valid_models]
                    if ms:
                        config['models'] = ms
                if 'reason' in parsed:
                    config['reason'] = str(parsed['reason'])
                if 'hyperparams' in parsed:
                    config['hyperparams'] = parsed['hyperparams']
            print(f"  [OK] Agent 推荐模型: {config['models']}")
            print(f"       理由: {config['reason']}")
        except Exception as e:
            print(f"  [WARN] Agent 返回解析失败: {e}, 使用默认配置")
    else:
        print("  [WARN] Agent 未返回有效响应, 使用默认配置")

    return config


def build_model(model_name: str, hyperparams: dict):
    """
    根据模型名称和超参数实例化 sklearn 分类器。

    Description:
        支持 LogisticRegression, RandomForest, GradientBoosting 三种模型。
        超参数通过字典传入，未指定的参数使用默认值。

    Parameters:
        model_name : str
            模型名称，标量字符串。
        hyperparams : dict
            超参数字典。

    Returns:
        model : sklearn estimator
            实例化后的分类器对象。
    """
    if model_name == 'LogisticRegression':
        params = {'max_iter': 1000, 'solver': 'lbfgs', 'random_state': 42}
        params.update({k: v for k, v in hyperparams.items()
                       if k in ['max_iter', 'C', 'solver', 'penalty']})
        return LogisticRegression(**params)
    elif model_name == 'RandomForest':
        params = {'n_estimators': 200, 'max_depth': 6, 'random_state': 42,
                  'n_jobs': -1}
        params.update({k: v for k, v in hyperparams.items()
                       if k in ['n_estimators', 'max_depth',
                                'min_samples_split', 'min_samples_leaf']})
        return RandomForestClassifier(**params)
    elif model_name == 'GradientBoosting':
        params = {'n_estimators': 200, 'max_depth': 4,
                  'learning_rate': 0.05, 'random_state': 42,
                  'subsample': 0.8}
        params.update({k: v for k, v in hyperparams.items()
                       if k in ['n_estimators', 'max_depth',
                                'learning_rate', 'subsample']})
        return GradientBoostingClassifier(**params)
    else:
        return LogisticRegression(max_iter=1000, random_state=42)


def train_and_evaluate(df: pd.DataFrame, features: list,
                         target: str, train_idx, test_idx,
                         model_name: str, model) -> dict:
    """
    用指定模型和特征集在时间划分的训练/测试集上训练并评估。

    Description:
        1. 提取 Top50 特征和标签，去除缺失值行。
        2. 标准化特征。
        3. 训练模型，在测试集上预测。
        4. 计算 AUC, accuracy, precision, recall, f1 五项指标。
        5. 检查数据泄漏: 验证训练集最大日期 < 测试集最小日期。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns)。
        features : list of str
            特征列名列表，长度一般为 50。
        target : str
            目标标签列名。
        train_idx : pd.Index
            训练集行索引。
        test_idx : pd.Index
            测试集行索引。
        model_name : str
            模型名称，用于日志输出。
        model : sklearn estimator
            实例化后的分类器。

    Returns:
        result : dict
            包含 model_name, AUC, accuracy, precision, recall, f1,
            n_train, n_test, leakage_check 等键。
    """
    # 准备数据
    X_train = df.loc[train_idx, features].values
    y_train = df.loc[train_idx, target].values
    X_test = df.loc[test_idx, features].values
    y_test = df.loc[test_idx, target].values

    # 去除含 NaN 的行
    valid_tr = ~(np.isnan(X_train).any(axis=1) | np.isnan(y_train))
    valid_te = ~(np.isnan(X_test).any(axis=1) | np.isnan(y_test))
    X_train, y_train = X_train[valid_tr], y_train[valid_tr]
    X_test, y_test = X_test[valid_te], y_test[valid_te]

    # 标准化
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    # 训练
    print(f"    训练 {model_name} (训练集: {len(X_train):,}, "
          f"测试集: {len(X_test):,})...")
    model.fit(X_train_s, y_train)

    # 预测
    y_pred = model.predict(X_test_s)
    y_proba = model.predict_proba(X_test_s)

    # 指标计算
    result = {'model_name': model_name}
    n_classes = len(model.classes_)
    if n_classes == 2:
        result['AUC'] = float(roc_auc_score(y_test, y_proba[:, 1]))
    else:
        try:
            result['AUC'] = float(roc_auc_score(
                y_test, y_proba, multi_class='ovr', average='macro'))
        except Exception:
            result['AUC'] = 0.5

    result['accuracy'] = float(accuracy_score(y_test, y_pred))
    result['precision'] = float(precision_score(
        y_test, y_pred, average='macro', zero_division=0))
    result['recall'] = float(recall_score(
        y_test, y_pred, average='macro', zero_division=0))
    result['f1'] = float(f1_score(
        y_test, y_pred, average='macro', zero_division=0))
    result['n_train'] = len(X_train)
    result['n_test'] = len(X_test)

    # 数据泄漏检查
    train_max_date = df.loc[train_idx, 'trade_date'].max()
    test_min_date = df.loc[test_idx, 'trade_date'].min()
    leakage_ok = (train_max_date < test_min_date)
    result['leakage_check'] = 'PASS' if leakage_ok else 'FAIL'
    result['train_max_date'] = str(train_max_date)
    result['test_min_date'] = str(test_min_date)

    return result, model


def plot_model_comparison(results: list, img_dir: str = 'images') -> None:
    """
    绘制多模型性能对比雷达图和柱状图。

    Description:
        图1: 各模型指标柱状对比图（AUC/Precision/Recall/F1）
        图2: 混淆矩阵概要（仅文字展示区间）

    Parameters:
        results : list of dict
            各模型评估结果列表。
        img_dir : str
            图片保存目录，默认 'images'。
    """
    metrics = ['AUC', 'accuracy', 'precision', 'recall', 'f1']
    model_names = [r['model_name'] for r in results]

    fig, ax = plt.subplots(figsize=(10, 5))

    x = np.arange(len(metrics))
    width = 0.8 / len(results)
    colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

    for i, res in enumerate(results):
        vals = [res.get(m, 0) for m in metrics]
        bars = ax.bar(x + i * width, vals, width,
                       label=res['model_name'], color=colors[i % len(colors)],
                       edgecolor='white')
        for bar, val in zip(bars, vals):
            ax.text(bar.get_x() + bar.get_width() / 2., bar.get_height(),
                    f'{val:.3f}', ha='center', va='bottom', fontsize=8)

    ax.set_xlabel('评价指标')
    ax.set_ylabel('得分')
    ax.set_title('Top50 特征 — 分类模型性能对比')
    ax.set_xticks(x + width * (len(results) - 1) / 2)
    ax.set_xticklabels(metrics)
    ax.legend()
    ax.set_ylim(0, 1.05)
    ax.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    fig_path = os.path.join(img_dir, 'model_comparison.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"[OK] 模型对比图已保存: {fig_path}")


# ---- 执行 ----
print("=" * 60)
print("步骤 5.2: 模型验证")
print("=" * 60 + "\n")

# 1) Agent 选择模型
model_config = ask_agent_model_selection(
    top50_features, TARGET_LABEL,
    n_train=len(train_idx), n_test=len(test_idx))

print(f"\n  推荐模型: {model_config['models']}")
print(f"  超参数配置:")
for mname, hp in model_config['hyperparams'].items():
    if mname in model_config['models']:
        print(f"    {mname}: {hp}")

# 2) 逐模型训练与评估
print(f"\n  使用 Top50 特征 ({len(top50_features)} 个) 训练分类模型...\n")
all_results = []
trained_models = {}

for mname in model_config['models']:
    hp = model_config['hyperparams'].get(mname, {})
    model = build_model(mname, hp)
    result, fitted_model = train_and_evaluate(
        df_cleaned, top50_features, TARGET_LABEL,
        train_idx, test_idx, mname, model)
    all_results.append(result)
    trained_models[mname] = fitted_model

    print(f"\n    [{mname}] 结果:")
    print(f"      AUC       = {result['AUC']:.4f}")
    print(f"      Accuracy  = {result['accuracy']:.4f}")
    print(f"      Precision = {result['precision']:.4f}")
    print(f"      Recall    = {result['recall']:.4f}")
    print(f"      F1        = {result['f1']:.4f}")
    print(f"      泄漏检查  = {result['leakage_check']} "
          f"(训练截止: {result['train_max_date']}, "
          f"测试起始: {result['test_min_date']})")

# 3) 模型对比可视化
plot_model_comparison(all_results, IMG_DIR)

# 4) 最优模型
best = max(all_results, key=lambda x: x['AUC'])
print(f"\n[OK] 最优模型: {best['model_name']} (AUC={best['AUC']:.4f})")

# 5) 保存结果
results_df = pd.DataFrame(all_results)
results_df.to_csv('model_validation_results.csv', index=False,
                    encoding='utf-8-sig')
print(f"[OK] 模型验证结果已保存: model_validation_results.csv")

# 6) 汇总输出
print(f"\n{'='*60}")
print(f"[FINAL] 自动特征工程系统完成")
print(f"{'='*60}")
print(f"  原始特征: {len(FEATURE_COLS)}")
print(f"  清理后有效: {len(active_features)}")
print(f"  最终筛选: {len(top50_features)} 个 Top50 特征")
print(f"  目标标签: {TARGET_LABEL}")
print(f"  最优模型: {best['model_name']}")
print(f"  最优 AUC: {best['AUC']:.4f}")
print(f"  数据泄漏: 全部 {'PASS' if all(r['leakage_check']=='PASS' for r in all_results) else 'FAIL'}")
print(f"{'='*60}")

步骤 5.2: 模型验证

  [AGENT] 正在请求千问Agent选择分类模型...


[2026-02-28 17:54:58,284] INFO: HTTP Request: POST https://api.siliconflow.cn/v1/chat/completions "HTTP/1.1 200 OK"


  [OK] Agent 推荐模型: ['LogisticRegression']
       理由: 推荐理由：在金融时序数据分类任务中，Logistic回归模型能够快速训练并且提供较高的可解释性，适合初步筛选特征和生成基准模型。XGBoost则是当前在排序问题和分类问题上表现优异的模型，且已经针对高效性和准确性进行了优化。

  推荐模型: ['LogisticRegression']
  超参数配置:
    LogisticRegression: {'max_iter': 1000, 'C': 1.0}

  使用 Top50 特征 (50 个) 训练分类模型...

    训练 LogisticRegression (训练集: 61,923, 测试集: 19,123)...

    [LogisticRegression] 结果:
      AUC       = 0.5689
      Accuracy  = 0.6999
      Precision = 0.3984
      Recall    = 0.3359
      F1        = 0.2835
      泄漏检查  = PASS (训练截止: 2019-10-22 00:00:00, 测试起始: 2019-10-23 00:00:00)
[OK] 模型对比图已保存: images\model_comparison.png

[OK] 最优模型: LogisticRegression (AUC=0.5689)
[OK] 模型验证结果已保存: model_validation_results.csv

[FINAL] 自动特征工程系统完成
  原始特征: 300
  清理后有效: 300
  最终筛选: 50 个 Top50 特征
  目标标签: Y1
  最优模型: LogisticRegression
  最优 AUC: 0.5689
  数据泄漏: 全部 PASS


## 步骤6：输出可视化与完整报告
- 6.1 综合可视化：数据概况（缺失热力图/时序折线图）、特征处理（清理前后对比/异常值箱型图）、特征评估（重要性条形图/冗余热力图）
- 6.2 模型评估可视化：混淆矩阵、ROC曲线、Precision-Recall曲线
- 6.3 数据泄漏检测报告：汇总步骤1~5中所有泄漏检查结果
- 6.4 完整结构化报告汇总：步骤1~6全过程决策、结果、关键输出索引

### 6.1 综合可视化
数据概况可视化（缺失值热力图、时序折线图）、特征清理前后对比（分布对比图、异常值箱型图）、特征评估可视化（Top50重要性条形图、冗余特征相关性热力图）。

In [20]:
# =============================================================================
# Cell: 6.1 综合可视化 — 数据概况 + 特征处理 + 特征评估
# =============================================================================


def plot_data_overview(df: pd.DataFrame, df_cleaned: pd.DataFrame,
                         top50: list, img_dir: str = 'images') -> None:
    """
    绘制数据概况可视化：缺失值热力图 + 收盘价时序折线图。

    Description:
        图1: Top50 特征的缺失值热力图 (原始数据)
        图2: 收盘价按日期的时序折线图，标注训练/测试分界线

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据。
        df_cleaned : pd.DataFrame
            形状为 (n_samples, n_columns) 的清理后数据。
        top50 : list of str
            Top50 特征名列表。
        img_dir : str
            图片保存目录，默认 'images'。
    """
    fig, axes = plt.subplots(1, 2, figsize=(20, 7))

    # 图1: 缺失值热力图 (原始数据, Top50 特征)
    ax1 = axes[0]
    missing_matrix = df[top50].isnull().astype(int)
    # 采样展示避免过大
    if len(missing_matrix) > 2000:
        sample_idx = np.linspace(0, len(missing_matrix)-1, 2000, dtype=int)
        missing_matrix = missing_matrix.iloc[sample_idx]
    sns.heatmap(missing_matrix.T, cbar=False, cmap='YlOrRd',
                yticklabels=True, ax=ax1)
    ax1.set_title('Top50 特征缺失值热力图 (原始数据)')
    ax1.set_xlabel('样本索引 (采样)')
    ax1.set_ylabel('特征')
    ax1.tick_params(axis='y', labelsize=6)

    # 图2: 收盘价时序折线图
    ax2 = axes[1]
    if 'trade_date' in df.columns and 'close' in df.columns:
        daily = df.groupby('trade_date')['close'].mean().sort_index()
        ax2.plot(range(len(daily)), daily.values, color='steelblue',
                 linewidth=0.8, alpha=0.8)
        # 标注训练/测试分界线
        split_pos = int(len(daily) * 0.8)
        ax2.axvline(x=split_pos, color='red', linestyle='--',
                    linewidth=1.5, label='训练/测试分界')
        ax2.set_xlabel('交易日序号')
        ax2.set_ylabel('收盘价 (日均)')
        ax2.set_title('收盘价时序趋势 (训练/测试划分)')
        ax2.legend(fontsize=9)
        ax2.grid(alpha=0.3)

    plt.tight_layout()
    fig_path = os.path.join(img_dir, 'step6_data_overview.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"[OK] 数据概况可视化已保存: {fig_path}")


def plot_feature_cleaning_comparison(df: pd.DataFrame,
                                       df_cleaned: pd.DataFrame,
                                       top50: list,
                                       img_dir: str = 'images') -> None:
    """
    绘制特征清理前后对比：分布对比直方图 + 异常值箱型图。

    Description:
        图1: 选取 Top50 中前 6 个特征, 清理前后分布直方图对比
        图2: 选取 Top50 中前 10 个特征, 清理前后箱型图对比

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据。
        df_cleaned : pd.DataFrame
            形状为 (n_samples, n_columns) 的清理后数据。
        top50 : list of str
            Top50 特征名列表。
        img_dir : str
            图片保存目录，默认 'images'。
    """
    # 图1: 分布对比直方图 (前 6 个特征)
    show_feats = [f for f in top50[:6] if f in df.columns and f in df_cleaned.columns]
    if show_feats:
        fig, axes = plt.subplots(2, len(show_feats), figsize=(4*len(show_feats), 8))
        if len(show_feats) == 1:
            axes = axes.reshape(2, 1)
        for j, feat in enumerate(show_feats):
            # 清理前
            ax_before = axes[0, j]
            vals_before = df[feat].dropna()
            ax_before.hist(vals_before, bins=50, color='#e74c3c', alpha=0.7,
                           edgecolor='white')
            ax_before.set_title(f'{feat} (清理前)', fontsize=9)
            ax_before.set_ylabel('频数' if j == 0 else '')
            ax_before.tick_params(labelsize=7)

            # 清理后
            ax_after = axes[1, j]
            vals_after = df_cleaned[feat].dropna()
            ax_after.hist(vals_after, bins=50, color='#2ecc71', alpha=0.7,
                          edgecolor='white')
            ax_after.set_title(f'{feat} (清理后)', fontsize=9)
            ax_after.set_ylabel('频数' if j == 0 else '')
            ax_after.tick_params(labelsize=7)

        plt.suptitle('Top50 特征清理前后分布对比', fontsize=13, y=1.01)
        plt.tight_layout()
        fig_path = os.path.join(img_dir, 'step6_distribution_compare.png')
        plt.savefig(fig_path, bbox_inches='tight')
        plt.close()
        print(f"[OK] 分布对比图已保存: {fig_path}")

    # 图2: 异常值箱型图 (前 10 个特征)
    show_box = [f for f in top50[:10] if f in df.columns and f in df_cleaned.columns]
    if show_box:
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        # 清理前
        ax1 = axes[0]
        box_data_before = [df[f].dropna().values for f in show_box]
        bp1 = ax1.boxplot(box_data_before, labels=show_box, patch_artist=True,
                           showfliers=True)
        for patch in bp1['boxes']:
            patch.set_facecolor('#e74c3c')
            patch.set_alpha(0.5)
        ax1.set_title('清理前 — 异常值箱型图 (Top10 特征)')
        ax1.tick_params(axis='x', rotation=45, labelsize=8)
        ax1.grid(axis='y', alpha=0.3)

        # 清理后
        ax2 = axes[1]
        box_data_after = [df_cleaned[f].dropna().values for f in show_box]
        bp2 = ax2.boxplot(box_data_after, labels=show_box, patch_artist=True,
                           showfliers=True)
        for patch in bp2['boxes']:
            patch.set_facecolor('#2ecc71')
            patch.set_alpha(0.5)
        ax2.set_title('清理后 — 异常值箱型图 (Top10 特征)')
        ax2.tick_params(axis='x', rotation=45, labelsize=8)
        ax2.grid(axis='y', alpha=0.3)

        plt.tight_layout()
        fig_path = os.path.join(img_dir, 'step6_outlier_boxplot.png')
        plt.savefig(fig_path, bbox_inches='tight')
        plt.close()
        print(f"[OK] 异常值箱型图已保存: {fig_path}")


def plot_feature_evaluation_viz(evaluation_report: pd.DataFrame,
                                  top50: list,
                                  df_cleaned: pd.DataFrame,
                                  img_dir: str = 'images') -> None:
    """
    绘制特征评估可视化：Top50重要性条形图 + 冗余特征相关性热力图。

    Description:
        图1: Top50 特征综合得分水平条形图（按 final_score 降序）
        图2: Top50 特征间皮尔逊相关系数热力图

    Parameters:
        evaluation_report : pd.DataFrame
            综合评估报告表。
        top50 : list of str
            Top50 特征名列表。
        df_cleaned : pd.DataFrame
            形状为 (n_samples, n_columns) 的清理后数据。
        img_dir : str
            图片保存目录，默认 'images'。
    """
    fig, axes = plt.subplots(1, 2, figsize=(20, 10))

    # 图1: Top50 特征重要性条形图
    ax1 = axes[0]
    top50_report = evaluation_report[
        evaluation_report['feature'].isin(top50)
    ].copy()
    top50_report = top50_report.sort_values('composite_score', ascending=True)
    colors_bar = []
    for _, r in top50_report.iterrows():
        if r.get('is_redundant', False) and r.get('is_multicollinear', False):
            colors_bar.append('#e74c3c')  # high risk
        elif r.get('is_redundant', False) or r.get('is_multicollinear', False):
            colors_bar.append('#f39c12')  # medium risk
        else:
            colors_bar.append('#2ecc71')  # low risk

    ax1.barh(range(len(top50_report)), top50_report['composite_score'].values,
             color=colors_bar, edgecolor='white', height=0.7)
    ax1.set_yticks(range(len(top50_report)))
    ax1.set_yticklabels(top50_report['feature'].values, fontsize=7)
    ax1.set_xlabel('综合得分 (composite_score)')
    ax1.set_title(f'Top50 特征重要性排名')
    from matplotlib.patches import Patch
    legend_items = [
        Patch(facecolor='#2ecc71', label='low risk'),
        Patch(facecolor='#f39c12', label='medium risk'),
        Patch(facecolor='#e74c3c', label='high risk')
    ]
    ax1.legend(handles=legend_items, loc='lower right', fontsize=8)

    # 图2: Top50 特征间相关性热力图
    ax2 = axes[1]
    corr_top50 = df_cleaned[top50].corr(method='pearson')
    mask = np.triu(np.ones_like(corr_top50, dtype=bool), k=1)
    sns.heatmap(corr_top50, mask=mask, cmap='RdBu_r', center=0,
                vmin=-1, vmax=1, square=True, linewidths=0.3,
                xticklabels=True, yticklabels=True, ax=ax2,
                cbar_kws={'shrink': 0.6})
    ax2.set_title('Top50 特征间相关性热力图')
    ax2.tick_params(axis='both', labelsize=5)

    plt.tight_layout()
    fig_path = os.path.join(img_dir, 'step6_feature_evaluation.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"[OK] 特征评估可视化已保存: {fig_path}")


# ---- 执行 ----
print("=" * 60)
print("步骤 6.1: 综合可视化")
print("=" * 60 + "\n")

# 数据概况
plot_data_overview(df, df_cleaned, top50_features, IMG_DIR)

# 特征清理前后对比
plot_feature_cleaning_comparison(df, df_cleaned, top50_features, IMG_DIR)

# 特征评估
plot_feature_evaluation_viz(evaluation_report, top50_features,
                              df_cleaned, IMG_DIR)

print("\n[OK] 步骤 6.1 综合可视化完成")

步骤 6.1: 综合可视化

[OK] 数据概况可视化已保存: images\step6_data_overview.png
[OK] 分布对比图已保存: images\step6_distribution_compare.png
[OK] 异常值箱型图已保存: images\step6_outlier_boxplot.png
[OK] 特征评估可视化已保存: images\step6_feature_evaluation.png

[OK] 步骤 6.1 综合可视化完成


### 6.2 模型评估可视化

- 混淆矩阵热力图
- ROC 曲线 (含 AUC 值标注)
- Precision-Recall 曲线

In [22]:
# =============================================================================
# Cell: 6.2 模型评估可视化 — 混淆矩阵 / ROC / PR 曲线
# =============================================================================

from sklearn.metrics import (roc_curve, precision_recall_curve,
                             average_precision_score, confusion_matrix,
                             roc_auc_score, classification_report)
from sklearn.preprocessing import label_binarize


def retrain_and_predict(df_cleaned: pd.DataFrame,
                         top50: list,
                         target: str,
                         train_idx: np.ndarray,
                         test_idx: np.ndarray) -> dict:
    """
    使用 Top50 特征重新训练 LogisticRegression 并返回预测结果。

    Description:
        对清理后数据进行标准化 -> 训练 LR -> 收集 y_true / y_pred / y_proba。
        自动检测二分类/多分类并使用对应 AUC 计算方式。

    Parameters:
        df_cleaned : pd.DataFrame
            形状为 (n_samples, n_columns) 的清理后数据。
        top50 : list of str
            长度为 50 的特征名列表。
        target : str
            目标标签列名。
        train_idx : np.ndarray
            训练集行索引。
        test_idx : np.ndarray
            测试集行索引。

    Returns:
        dict : 包含 y_train, y_test, y_pred, y_proba, model, scaler,
               auc, n_classes, classes 的字典。
    """
    X_train = df_cleaned.loc[train_idx, top50].values
    X_test = df_cleaned.loc[test_idx, top50].values
    y_train = df_cleaned.loc[train_idx, target].values
    y_test = df_cleaned.loc[test_idx, target].values

    # 去除含 NaN 的行
    valid_tr = ~(np.isnan(X_train).any(axis=1) | np.isnan(y_train))
    valid_te = ~(np.isnan(X_test).any(axis=1) | np.isnan(y_test))
    X_train, y_train = X_train[valid_tr], y_train[valid_tr]
    X_test, y_test = X_test[valid_te], y_test[valid_te]

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    model = LogisticRegression(max_iter=1000, random_state=42,
                               class_weight='balanced', solver='lbfgs')
    model.fit(X_train_s, y_train)

    y_pred = model.predict(X_test_s)
    y_proba = model.predict_proba(X_test_s)

    n_classes = len(model.classes_)
    if n_classes == 2:
        auc = roc_auc_score(y_test, y_proba[:, 1])
    else:
        try:
            auc = roc_auc_score(y_test, y_proba,
                                multi_class='ovr', average='macro')
        except Exception:
            auc = 0.5

    print(f"[OK] 重新训练完成 | n_classes={n_classes} | AUC = {auc:.4f} "
          f"| 测试集样本 = {len(y_test)}")
    return {
        'y_train': y_train, 'y_test': y_test,
        'y_pred': y_pred, 'y_proba': y_proba,
        'model': model, 'scaler': scaler, 'auc': auc,
        'n_classes': n_classes, 'classes': model.classes_,
        'X_train_s': X_train_s, 'X_test_s': X_test_s
    }


def plot_confusion_matrix_heatmap(y_test, y_pred, classes, ax=None):
    """
    绘制混淆矩阵热力图 (支持多分类)。

    Parameters:
        y_test : np.ndarray, 形状 (n_test,), 真实标签。
        y_pred : np.ndarray, 形状 (n_test,), 预测标签。
        classes : np.ndarray, 类别标签数组。
        ax : matplotlib.axes.Axes or None, 绘图轴。
    """
    cm = confusion_matrix(y_test, y_pred, labels=classes)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=[f'{c}' for c in classes],
                yticklabels=[f'{c}' for c in classes])
    ax.set_title('混淆矩阵')
    ax.set_xlabel('预测标签')
    ax.set_ylabel('真实标签')


def plot_roc_multiclass(y_test, y_proba, classes, auc_val, ax=None):
    """
    绘制 ROC 曲线 (多分类: 每类 OVR + macro 均值)。

    Parameters:
        y_test : np.ndarray, 形状 (n_test,), 真实标签。
        y_proba : np.ndarray, 形状 (n_test, n_classes), 各类概率。
        classes : np.ndarray, 类别标签数组。
        auc_val : float, macro-average AUC。
        ax : matplotlib.axes.Axes or None, 绘图轴。
    """
    y_bin = label_binarize(y_test, classes=classes)
    n_classes = len(classes)
    colors = plt.cm.Set2(np.linspace(0, 1, max(n_classes, 3)))

    if n_classes == 2:
        fpr, tpr, _ = roc_curve(y_test, y_proba[:, 1])
        ax.plot(fpr, tpr, color=colors[0], linewidth=2,
                label=f'ROC (AUC = {auc_val:.4f})')
        ax.fill_between(fpr, tpr, alpha=0.1, color=colors[0])
    else:
        for i, cls in enumerate(classes):
            if y_bin.ndim == 1:
                y_i = (y_test == cls).astype(int)
            else:
                y_i = y_bin[:, i]
            fpr_i, tpr_i, _ = roc_curve(y_i, y_proba[:, i])
            try:
                cls_auc = roc_auc_score(y_i, y_proba[:, i])
            except Exception:
                cls_auc = 0.5
            ax.plot(fpr_i, tpr_i, color=colors[i % len(colors)],
                    linewidth=1.5,
                    label=f'Class {cls} (AUC={cls_auc:.3f})')

    ax.plot([0, 1], [0, 1], linestyle='--', color='gray',
            linewidth=1, alpha=0.7)
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'ROC 曲线 (macro AUC={auc_val:.4f})')
    ax.legend(loc='lower right', fontsize=7)
    ax.grid(alpha=0.3)


def plot_pr_multiclass(y_test, y_proba, classes, ax=None):
    """
    绘制 Precision-Recall 曲线 (多分类: 每类 OVR)。

    Parameters:
        y_test : np.ndarray, 形状 (n_test,), 真实标签。
        y_proba : np.ndarray, 形状 (n_test, n_classes), 各类概率。
        classes : np.ndarray, 类别标签数组。
        ax : matplotlib.axes.Axes or None, 绘图轴。
    """
    y_bin = label_binarize(y_test, classes=classes)
    n_classes = len(classes)
    colors = plt.cm.Set2(np.linspace(0, 1, max(n_classes, 3)))

    if n_classes == 2:
        precision, recall, _ = precision_recall_curve(y_test, y_proba[:, 1])
        ap = average_precision_score(y_test, y_proba[:, 1])
        ax.plot(recall, precision, color=colors[0], linewidth=2,
                label=f'PR (AP={ap:.4f})')
        ax.fill_between(recall, precision, alpha=0.1, color=colors[0])
    else:
        for i, cls in enumerate(classes):
            if y_bin.ndim == 1:
                y_i = (y_test == cls).astype(int)
            else:
                y_i = y_bin[:, i]
            prec_i, rec_i, _ = precision_recall_curve(y_i, y_proba[:, i])
            try:
                ap_i = average_precision_score(y_i, y_proba[:, i])
            except Exception:
                ap_i = 0.0
            ax.plot(rec_i, prec_i, color=colors[i % len(colors)],
                    linewidth=1.5,
                    label=f'Class {cls} (AP={ap_i:.3f})')

    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_title('Precision-Recall 曲线')
    ax.legend(loc='upper right', fontsize=7)
    ax.grid(alpha=0.3)


# ---- 执行 ----
print("=" * 60)
print("步骤 6.2: 模型评估可视化")
print("=" * 60 + "\n")

# 重新训练获取预测结果
pred_results = retrain_and_predict(df_cleaned, top50_features,
                                     TARGET_LABEL, train_idx, test_idx)

# 绘制三合一图
fig, axes = plt.subplots(1, 3, figsize=(21, 6))

plot_confusion_matrix_heatmap(pred_results['y_test'], pred_results['y_pred'],
                                pred_results['classes'], ax=axes[0])
plot_roc_multiclass(pred_results['y_test'], pred_results['y_proba'],
                      pred_results['classes'], pred_results['auc'],
                      ax=axes[1])
plot_pr_multiclass(pred_results['y_test'], pred_results['y_proba'],
                     pred_results['classes'], ax=axes[2])

plt.suptitle(f'模型评估 -- LogisticRegression | 特征数={len(top50_features)} '
             f'| 类别数={pred_results["n_classes"]}',
             fontsize=14, y=1.02)
plt.tight_layout()
fig_path = os.path.join(IMG_DIR, 'step6_model_evaluation.png')
plt.savefig(fig_path, bbox_inches='tight')
plt.close()
print(f"[OK] 模型评估可视化已保存: {fig_path}")

# 分类报告
print("\n--- 分类报告 ---")
print(classification_report(pred_results['y_test'], pred_results['y_pred'],
                              digits=4))

print("\n[OK] 步骤 6.2 模型评估可视化完成")

步骤 6.2: 模型评估可视化

[OK] 重新训练完成 | n_classes=3 | AUC = 0.5671 | 测试集样本 = 19123
[OK] 模型评估可视化已保存: images\step6_model_evaluation.png

--- 分类报告 ---
              precision    recall  f1-score   support

        -1.0     0.7553    0.5625    0.6448     13423
         0.0     0.1633    0.3415    0.2209      2662
         1.0     0.1927    0.2258    0.2079      3038

    accuracy                         0.4782     19123
   macro avg     0.3704    0.3766    0.3579     19123
weighted avg     0.5835    0.4782    0.5164     19123


[OK] 步骤 6.2 模型评估可视化完成


### 6.3 数据泄漏检测报告 (独立模块)

独立汇总全流程中的数据泄漏检测结果：
1. 时间序列划分验证：训练集日期 < 测试集日期
2. 特征泄漏检测：检查是否有未来信息泄漏
3. 模型泄漏检测：训练/测试 AUC 差异是否异常

In [24]:
# =============================================================================
# Cell: 6.3 数据泄漏检测报告 — 独立模块
# =============================================================================


def run_leakage_detection(df: pd.DataFrame,
                            df_cleaned: pd.DataFrame,
                            train_idx: np.ndarray,
                            test_idx: np.ndarray,
                            top50: list,
                            target: str,
                            pred_results: dict,
                            img_dir: str = 'images') -> pd.DataFrame:
    """
    独立数据泄漏检测模块，汇总全流程泄漏风险。

    Description:
        检测维度:
        1. 时间序列划分完整性: 训练集最大日期 < 测试集最小日期
        2. 样本泄漏: 训练集与测试集是否有重叠行
        3. 特征泄漏 (标签相关性): Top50 中是否有与标签相关性过高 (|r|>0.95) 的特征
        4. 模型泄漏 (过拟合检测): train_AUC - test_AUC 差值是否 > 0.15
        5. 标签分布一致性: 训练/测试正样本率差异是否 < 0.10

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据。
        df_cleaned : pd.DataFrame
            形状为 (n_samples, n_columns) 的清理后数据。
        train_idx : np.ndarray
            训练集行索引。
        test_idx : np.ndarray
            测试集行索引。
        top50 : list of str
            长度为 50 的特征名列表。
        target : str
            目标标签列名。
        pred_results : dict
            包含 model, scaler, auc, y_test, y_train, n_classes 等。
        img_dir : str
            图片保存目录。

    Returns:
        pd.DataFrame : 形状为 (n_checks, 4) 的泄漏检测报告，
                       列 = ['检测项', '结果', '详情', '风险等级']。
    """
    checks = []

    # ------------------------------------------------------------------
    # 1. 时间序列划分验证
    # ------------------------------------------------------------------
    if 'trade_date' in df.columns:
        train_max = df.loc[train_idx, 'trade_date'].max()
        test_min = df.loc[test_idx, 'trade_date'].min()
        time_pass = train_max < test_min
        checks.append({
            '检测项': '时间序列划分',
            '结果': 'PASS' if time_pass else 'FAIL',
            '详情': f'训练集最大日期={train_max}, 测试集最小日期={test_min}',
            '风险等级': '无风险' if time_pass else '高风险'
        })
    else:
        checks.append({
            '检测项': '时间序列划分',
            '结果': 'SKIP',
            '详情': '未找到 trade_date 列',
            '风险等级': '未知'
        })

    # ------------------------------------------------------------------
    # 2. 样本泄漏 (训练/测试集重叠)
    # ------------------------------------------------------------------
    overlap = set(train_idx) & set(test_idx)
    sample_pass = len(overlap) == 0
    checks.append({
        '检测项': '样本泄漏 (行重叠)',
        '结果': 'PASS' if sample_pass else 'FAIL',
        '详情': f'重叠样本数={len(overlap)}',
        '风险等级': '无风险' if sample_pass else '高风险'
    })

    # ------------------------------------------------------------------
    # 3. 特征泄漏 (极高标签相关性)
    # ------------------------------------------------------------------
    high_corr_feats = []
    for feat in top50:
        if feat in df_cleaned.columns and target in df_cleaned.columns:
            valid = df_cleaned[[feat, target]].dropna()
            if len(valid) > 30:
                corr_val = abs(valid[feat].corr(valid[target]))
                if corr_val > 0.95:
                    high_corr_feats.append((feat, corr_val))

    feat_pass = len(high_corr_feats) == 0
    detail_str = '无' if feat_pass else '; '.join(
        [f'{f}(|r|={v:.3f})' for f, v in high_corr_feats])
    checks.append({
        '检测项': '特征泄漏 (|r|>0.95)',
        '结果': 'PASS' if feat_pass else 'WARN',
        '详情': f'极高相关特征: {detail_str}',
        '风险等级': '无风险' if feat_pass else '高风险'
    })

    # ------------------------------------------------------------------
    # 4. 模型泄漏 (过拟合检测)
    # ------------------------------------------------------------------
    model = pred_results['model']
    scaler = pred_results['scaler']
    n_classes = pred_results.get('n_classes', 2)

    # 使用已存储的训练数据重新预测
    X_train_raw = df_cleaned.loc[train_idx, top50].values
    y_train_raw = df_cleaned.loc[train_idx, target].values
    valid_tr = ~(np.isnan(X_train_raw).any(axis=1) | np.isnan(y_train_raw))
    X_train_valid = X_train_raw[valid_tr]
    y_train_valid = y_train_raw[valid_tr]
    X_train_s = scaler.transform(X_train_valid)
    train_proba = model.predict_proba(X_train_s)

    if n_classes == 2:
        train_auc = roc_auc_score(y_train_valid, train_proba[:, 1])
    else:
        try:
            train_auc = roc_auc_score(y_train_valid, train_proba,
                                        multi_class='ovr', average='macro')
        except Exception:
            train_auc = 0.5

    test_auc = pred_results['auc']
    auc_gap = train_auc - test_auc
    overfit_pass = auc_gap < 0.15
    checks.append({
        '检测项': '模型泄漏 (过拟合)',
        '结果': 'PASS' if overfit_pass else 'WARN',
        '详情': f'train_AUC={train_auc:.4f}, test_AUC={test_auc:.4f}, gap={auc_gap:.4f}',
        '风险等级': '无风险' if overfit_pass else '中风险'
    })

    # ------------------------------------------------------------------
    # 5. 标签分布一致性
    # ------------------------------------------------------------------
    from collections import Counter
    train_dist = Counter(y_train_valid)
    test_dist = Counter(pred_results['y_test'])
    train_total = sum(train_dist.values())
    test_total = sum(test_dist.values())

    all_classes = sorted(set(list(train_dist.keys()) + list(test_dist.keys())))
    dist_details = []
    max_diff = 0.0
    for cls in all_classes:
        tr_rate = train_dist.get(cls, 0) / train_total
        te_rate = test_dist.get(cls, 0) / test_total
        diff = abs(tr_rate - te_rate)
        max_diff = max(max_diff, diff)
        dist_details.append(f'class={cls}: train={tr_rate:.3f}, test={te_rate:.3f}')

    dist_pass = max_diff < 0.10
    checks.append({
        '检测项': '标签分布一致性',
        '结果': 'PASS' if dist_pass else 'WARN',
        '详情': '; '.join(dist_details) + f' | max_diff={max_diff:.4f}',
        '风险等级': '无风险' if dist_pass else '中风险'
    })

    report_df = pd.DataFrame(checks)
    return report_df


# ---- 执行 ----
print("=" * 60)
print("步骤 6.3: 数据泄漏检测报告 (独立模块)")
print("=" * 60 + "\n")

leakage_detection_report = run_leakage_detection(
    df, df_cleaned, train_idx, test_idx,
    top50_features, TARGET_LABEL, pred_results, IMG_DIR
)

# 打印报告
print("--- 数据泄漏检测汇总 ---\n")
for _, row in leakage_detection_report.iterrows():
    status_tag = '[PASS]' if row['结果'] == 'PASS' else (
        '[WARN]' if row['结果'] == 'WARN' else '[FAIL]')
    print(f"  {status_tag} {row['检测项']}")
    print(f"       详情: {row['详情']}")
    print(f"       风险等级: {row['风险等级']}\n")

# 保存
leakage_csv = 'leakage_detection_report.csv'
leakage_detection_report.to_csv(leakage_csv, index=False, encoding='utf-8-sig')
print(f"[OK] 泄漏检测报告已保存: {leakage_csv}")

# 总结
n_pass = (leakage_detection_report['结果'] == 'PASS').sum()
n_total = len(leakage_detection_report)
print(f"\n[SUMMARY] 通过 {n_pass}/{n_total} 项检测")
if n_pass == n_total:
    print("[OK] 全部检测通过, 无数据泄漏风险")
else:
    print("[WARN] 存在部分风险项, 请关注上述详情")

print("\n[OK] 步骤 6.3 数据泄漏检测报告完成")

步骤 6.3: 数据泄漏检测报告 (独立模块)

--- 数据泄漏检测汇总 ---

  [PASS] 时间序列划分
       详情: 训练集最大日期=2019-10-22 00:00:00, 测试集最小日期=2019-10-23 00:00:00
       风险等级: 无风险

  [PASS] 样本泄漏 (行重叠)
       详情: 重叠样本数=0
       风险等级: 无风险

  [PASS] 特征泄漏 (|r|>0.95)
       详情: 极高相关特征: 无
       风险等级: 无风险

  [PASS] 模型泄漏 (过拟合)
       详情: train_AUC=0.6182, test_AUC=0.5671, gap=0.0511
       风险等级: 无风险

  [PASS] 标签分布一致性
       详情: class=-1.0: train=0.724, test=0.702; class=0.0: train=0.135, test=0.139; class=1.0: train=0.141, test=0.159 | max_diff=0.0221
       风险等级: 无风险

[OK] 泄漏检测报告已保存: leakage_detection_report.csv

[SUMMARY] 通过 5/5 项检测
[OK] 全部检测通过, 无数据泄漏风险

[OK] 步骤 6.3 数据泄漏检测报告完成


### 6.4 完整报告汇总

汇总 Steps 1~6 全流程结果，生成结构化报告：
- 各步骤决策摘要、关键指标
- 产出文件索引
- 系统设计总结

In [25]:
# =============================================================================
# Cell: 6.4 完整报告汇总
# =============================================================================


def generate_final_report(df: pd.DataFrame,
                            df_cleaned: pd.DataFrame,
                            top50: list,
                            evaluation_report: pd.DataFrame,
                            pred_results: dict,
                            leakage_detection_report: pd.DataFrame,
                            target: str) -> str:
    """
    生成全流程结构化报告文本。

    Description:
        汇总 Steps 1~6 的关键决策、指标、产出文件索引。

    Parameters:
        df : pd.DataFrame
            形状为 (n_samples, n_columns) 的原始数据。
        df_cleaned : pd.DataFrame
            形状为 (n_samples, n_columns) 的清理后数据。
        top50 : list of str
            长度为 50 的最终选定特征列表。
        evaluation_report : pd.DataFrame
            特征综合评估报告。
        pred_results : dict
            模型预测结果字典。
        leakage_detection_report : pd.DataFrame
            泄漏检测报告。
        target : str
            目标标签列名。

    Returns:
        str : 完整报告文本。
    """
    sep = "=" * 70
    lines = []
    lines.append(sep)
    lines.append("  自动化特征工程系统 -- 全流程报告")
    lines.append(sep)
    lines.append("")

    # ---- Step 1 ----
    lines.append("-" * 50)
    lines.append("Step 1: 数据初始化与验证")
    lines.append("-" * 50)
    lines.append(f"  数据文件: data.pq")
    lines.append(f"  原始维度: {df.shape[0]} 行 x {df.shape[1]} 列")
    n_features = len([c for c in df.columns if c.startswith('X')])
    n_labels = len([c for c in df.columns if c.startswith('Y')])
    lines.append(f"  特征列: {n_features} 个 (X1~X{n_features})")
    lines.append(f"  标签列: {n_labels} 个")
    if 'trade_date' in df.columns:
        lines.append(f"  日期范围: {df['trade_date'].min()} ~ {df['trade_date'].max()}")
    miss_pct = df[[c for c in df.columns if c.startswith('X')]].isnull().mean().mean() * 100
    lines.append(f"  平均缺失率: {miss_pct:.2f}%")
    lines.append("")

    # ---- Step 2 ----
    lines.append("-" * 50)
    lines.append("Step 2: 特征诊断")
    lines.append("-" * 50)
    lines.append(f"  诊断特征数: {n_features}")
    lines.append(f"  诊断维度: 缺失率, 零值率, 偏度, 峰度, ADF平稳性,")
    lines.append(f"             Shapiro-Wilk正态性, 异常值比例 (IQR)")
    lines.append(f"  产出: feature_diagnosis_report.csv")
    lines.append("")

    # ---- Step 3 ----
    lines.append("-" * 50)
    lines.append("Step 3: Agent 特征清理")
    lines.append("-" * 50)
    miss_before = df[[c for c in df.columns if c.startswith('X')]].isnull().mean().mean() * 100
    miss_after = df_cleaned[top50].isnull().mean().mean() * 100
    lines.append(f"  清理策略: Qwen Agent 分批决策 + 规则引擎兜底")
    lines.append(f"  缺失率变化: {miss_before:.2f}% -> {miss_after:.2f}%")
    lines.append(f"  产出: cleaning_plans.csv, cleaning_comparison.csv")
    lines.append("")

    # ---- Step 4 ----
    lines.append("-" * 50)
    lines.append("Step 4: 特征评估")
    lines.append("-" * 50)
    lines.append(f"  目标标签: {target}")
    lines.append(f"  评估指标: AUC (单特征LR) + |coef| 绝对值")
    lines.append(f"  划分方式: 按时间 80/20 分割")
    n_redundant = evaluation_report['is_redundant'].sum() if 'is_redundant' in evaluation_report.columns else 0
    n_multicol = evaluation_report['is_multicollinear'].sum() if 'is_multicollinear' in evaluation_report.columns else 0
    lines.append(f"  冗余特征对: {n_redundant} 个标记冗余")
    lines.append(f"  多重共线性: {n_multicol} 个 VIF>10")
    lines.append(f"  产出: feature_effectiveness.csv, redundant_pairs.csv, feature_evaluation_report.csv")
    lines.append("")

    # ---- Step 5 ----
    lines.append("-" * 50)
    lines.append("Step 5: 特征选择 + 模型验证")
    lines.append("-" * 50)
    lines.append(f"  筛选策略: 3轮过滤 (缺失>80%剔除 -> 冗余对剔除 -> 综合评分Top50)")
    lines.append(f"  评分公式: final_score = composite_norm*0.5 + label_corr_norm*0.3 + vif_bonus*0.2")
    lines.append(f"  最终特征数: {len(top50)}")
    lines.append(f"  模型: LogisticRegression (class_weight=balanced)")
    lines.append(f"  测试集 AUC: {pred_results['auc']:.4f}")
    lines.append(f"  产出: feature_selection_log.csv, model_validation_results.csv")
    lines.append("")

    # ---- Step 6 ----
    lines.append("-" * 50)
    lines.append("Step 6: 可视化与报告")
    lines.append("-" * 50)
    lines.append("  6.1 综合可视化:")
    lines.append("      - 缺失值热力图, 收盘价时序趋势")
    lines.append("      - 特征清理前后分布对比 (直方图)")
    lines.append("      - 异常值箱型图 (清理前后)")
    lines.append("      - Top50 特征重要性条形图")
    lines.append("      - Top50 特征间相关性热力图")
    lines.append("  6.2 模型评估可视化:")
    lines.append("      - 混淆矩阵")
    lines.append("      - ROC 曲线 (AUC 标注)")
    lines.append("      - Precision-Recall 曲线 (AP 标注)")
    lines.append("  6.3 数据泄漏检测:")
    n_leak_pass = (leakage_detection_report['结果'] == 'PASS').sum()
    n_leak_total = len(leakage_detection_report)
    lines.append(f"      通过 {n_leak_pass}/{n_leak_total} 项检测")
    lines.append("  6.4 完整报告: 本文档")
    lines.append("")

    # ---- 产出文件索引 ----
    lines.append(sep)
    lines.append("  产出文件索引")
    lines.append(sep)
    files_index = [
        ('feature_diagnosis_report.csv', 'Step 2 特征诊断报告'),
        ('cleaning_plans.csv', 'Step 3 Agent 清理方案'),
        ('cleaning_comparison.csv', 'Step 3 清理前后对比'),
        ('feature_effectiveness.csv', 'Step 4 单特征有效性'),
        ('redundant_pairs.csv', 'Step 4 冗余特征对'),
        ('feature_evaluation_report.csv', 'Step 4 综合评估报告'),
        ('feature_selection_log.csv', 'Step 5 特征筛选日志'),
        ('model_validation_results.csv', 'Step 5 模型验证结果'),
        ('leakage_detection_report.csv', 'Step 6 泄漏检测报告'),
        ('final_report.txt', 'Step 6 完整报告 (本文件)'),
    ]
    for fname, desc in files_index:
        marker = '[EXISTS]' if os.path.exists(fname) else '[MISSING]'
        lines.append(f"  {marker} {fname:40s} => {desc}")
    lines.append("")

    # ---- 图片索引 ----
    lines.append(sep)
    lines.append("  可视化图片索引")
    lines.append(sep)
    img_files = [
        'step6_data_overview.png',
        'step6_distribution_compare.png',
        'step6_outlier_boxplot.png',
        'step6_feature_evaluation.png',
        'step6_model_evaluation.png',
        'close_price_trend.png',
        'missing_heatmap.png',
        'missing_bar.png',
        'label_distributions_all.png',
        'correlation_distribution.png',
        'missing_top20.png',
        'outlier_boxplot_top20.png',
        'feature_label_corr_heatmap.png',
        'missing_before_after.png',
        'outlier_before_after.png',
        'feature_evaluation_summary.png',
        'model_comparison.png',
    ]
    for img in img_files:
        path = os.path.join('images', img)
        marker = '[EXISTS]' if os.path.exists(path) else '[MISSING]'
        lines.append(f"  {marker} {path}")
    lines.append("")

    # ---- Top50 特征列表 ----
    lines.append(sep)
    lines.append("  Top50 最终特征列表")
    lines.append(sep)
    for i, feat in enumerate(top50, 1):
        lines.append(f"  {i:3d}. {feat}")
    lines.append("")

    # ---- 系统设计总结 ----
    lines.append(sep)
    lines.append("  Agent 系统设计总结")
    lines.append(sep)
    lines.append("  1. 架构: Pipeline 式 Agent (数据初始化 -> 诊断 -> 清理 -> 评估 -> 选择 -> 报告)")
    lines.append("  2. LLM 调用: SiliconFlow API (Qwen2.5-7B-Instruct)")
    lines.append("     - 使用 openai SDK 兼容接口")
    lines.append("     - System Prompt 约束 JSON 输出格式")
    lines.append("     - 分批处理 (30特征/批) 避免 token 溢出")
    lines.append("  3. 错误处理:")
    lines.append("     - JSON 解析失败 -> 正则回退提取")
    lines.append("     - Agent 动作不合法 -> 规则引擎兜底 (基于缺失率/异常值率)")
    lines.append("     - API 超时/异常 -> 重试 + 降级策略")
    lines.append("  4. 评估体系:")
    lines.append("     - 单特征 LogisticRegression AUC + 系数绝对值")
    lines.append("     - 冗余检测: Pearson |r|>0.8 + VIF")
    lines.append("     - 综合评分 = composite_norm*0.5 + label_corr_norm*0.3 + vif_bonus*0.2")
    lines.append("  5. 泄漏防控:")
    lines.append("     - 严格按时间划分训练/测试集")
    lines.append("     - 独立泄漏检测模块 (5 个维度)")
    lines.append("     - 特征与标签极高相关性告警")
    lines.append("")
    lines.append(sep)
    lines.append("  报告结束")
    lines.append(sep)

    return '\n'.join(lines)


# ---- 执行 ----
print("=" * 60)
print("步骤 6.4: 完整报告汇总")
print("=" * 60 + "\n")

final_report_text = generate_final_report(
    df, df_cleaned, top50_features, evaluation_report,
    pred_results, leakage_detection_report, TARGET_LABEL
)

# 打印报告
print(final_report_text)

# 保存报告
report_path = 'final_report.txt'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(final_report_text)
print(f"\n[OK] 完整报告已保存: {report_path}")

print("\n[OK] 步骤 6.4 完整报告汇总完成")
print("\n" + "=" * 60)
print("[OK] Step 6 全部完成")
print("=" * 60)

步骤 6.4: 完整报告汇总

  自动化特征工程系统 -- 全流程报告

--------------------------------------------------
Step 1: 数据初始化与验证
--------------------------------------------------
  数据文件: data.pq
  原始维度: 81046 行 x 321 列
  特征列: 300 个 (X1~X300)
  标签列: 12 个
  日期范围: 2015-01-05 00:00:00 ~ 2020-12-31 00:00:00
  平均缺失率: 23.64%

--------------------------------------------------
Step 2: 特征诊断
--------------------------------------------------
  诊断特征数: 300
  诊断维度: 缺失率, 零值率, 偏度, 峰度, ADF平稳性,
             Shapiro-Wilk正态性, 异常值比例 (IQR)
  产出: feature_diagnosis_report.csv

--------------------------------------------------
Step 3: Agent 特征清理
--------------------------------------------------
  清理策略: Qwen Agent 分批决策 + 规则引擎兜底
  缺失率变化: 23.64% -> 0.00%
  产出: cleaning_plans.csv, cleaning_comparison.csv

--------------------------------------------------
Step 4: 特征评估
--------------------------------------------------
  目标标签: Y1
  评估指标: AUC (单特征LR) + |coef| 绝对值
  划分方式: 按时间 80/20 分割
  冗余特征对: 139 个标记冗余
  多重共线性: 113 个 VIF>10
  产出: feat